Imports

In [105]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB
import numpy as np
import random
import pandas as pd
import math
# import scipy.sparse as sp

In [106]:
input = "input.xlsm"
# input = "https://umontevideo-my.sharepoint.com/personal/svilardelvalle_um_edu_uy/Documents/Horarios_2025/input.xlsm"
output = "output.xlsx"


In [107]:
# clases
from entities import Dia, Horario, BloqueHorario, Profesor, Materia, Grupo, Prioridad, Superposicion

In [108]:
# variables
from variables import u, v, w, x, y, z

# Funciones

## Cargar datos

In [109]:
# cargar datos
"""
This module provides functions to manage and manipulate data related to 'materias', 'grupos', and 'profesores'.

Functions:
    add_materia(materias, id, nombre, carga_horaria, cantidad_dias, grupos=[], profesores=[], cantidad_profesores=1):
        Adds a new 'materia' to the list of 'materias'.
        
    add_grupo(grupos, anio, turno, carrera, particion, recurse, aux):
        Adds a new 'grupo' to the list of 'grupos'.
        
    add_profesor(profesores, id, nombre, min_max_dias=None):
        Adds a new 'profesor' to the list of 'profesores'.
        
    lista_profesores(profesores, nombres):
        Returns a list of 'profesores' matching the given names.
        
    lista_grupos(grupos, nombres):
        Returns a list of 'grupos' matching the given names.
"""

# crear materia
def add_materia(materias, id, nombre, carga_horaria, cantidad_dias, grupos=[], profesores=[], cantidad_profesores=1, electiva=False, teo_prac=None):
    # id = len(materias)
    try:
        carga_horaria = int(carga_horaria)
    except:
        carga_horaria = 0
    try:
        cantidad_dias = int(cantidad_dias)
    except:
        cantidad_dias = 0
    materias.append(Materia(id, nombre, carga_horaria=carga_horaria, cantidad_dias=cantidad_dias,
                            grupos=grupos, profesores=profesores, cantidad_profesores=cantidad_profesores,
                            electiva=electiva, teo_prac=teo_prac))

# crear grupo
def add_grupo(grupos, anio, turno, carrera, particion, recurse, aux):
    id = len(grupos)
    grupos.append(Grupo(id, anio, turno, carrera, particion, bool(recurse), bool(aux)))

def grupos_reales(grupos): # no considerar grupos ficticios ("dummy") o con carga horaria nula (a proposito)

    return [g for g in grupos if not g.aux]

    # grupos_reales = []
    # for g in grupos:
    #     carga_horaria = 0
    #     for m in materias_grupo(g, materias):
    #         carga_horaria += m.carga_horaria
    #     if carga_horaria > 0:
    #         grupos_reales.append(g)

    # return grupos_reales

# crear profesor
def add_profesor(profesores, id, nombre, min_max_dias=None, nombre_completo=None):
    # id = len(profesores)
    profesor = Profesor(id, nombre, min_max_dias, nombre_completo)
    if not profesor in profesores:
        profesores.append(profesor)

def lista_profesores(profesores, nombres):
    profs = []
    for n in nombres:
        for p in profesores:
            if str(p) == n:
                profs.append(p)
                break
    return profs

def lista_grupos(grupos, nombres):
    gs = []
    for n in nombres:
        for g in grupos:
            if str(g) == n:
                gs.append(g)
                break
    return gs


In [110]:
#prioridades

"""
Generates a fixed priority array for the given time blocks with the specified priority value.
Args:
    bloques_horario (list): A list of time blocks.
    value (int): The fixed priority value to be assigned to each time block.
Returns:
    list: A list of lists where each inner list contains a time block and the specified priority value.
"""


def update_no_disp(profesor, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in profesor.no_disponible:
            profesor.no_disponible.append(bloques_horario[i])
"""
Updates the 'no_disponible' list of a professor by adding the specified time blocks.
Args:
    profesor (Profesor): The professor whose availability is being updated.
    bloques_horario (list): A list of time blocks.
    no_disp_index (list): A list of indices indicating which time blocks the professor is not available for.
"""


def update_prioridad(profesor, bloques_horario, array_prioridad):

    if profesor is None: return
    
    # reset:
    profesor.prioridades = []
    profesor.no_disponible = []

    """
    Updates the priority list of a professor based on the given priority array.
    Args:
        profesor (Profesor): The professor whose priorities are being updated.
        bloques_horario (list): A list of time blocks.
        array_prioridad (list): A list of tuples where each tuple contains a time block index and a priority value.
    """
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp(profesor, bloques_horario, [b_id])
        
        prior = Prioridad(value, bloques_horario[b_id], profesor = profesor)
        if not (prior in profesor.prioridades):
            profesor.prioridades.append(prior)

def random_pr(bloques_horario):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, random.randint(0,3)])
    return pr_array
"""
Generates a random priority array for the given time blocks.
Args:
    bloques_horario (list): A list of time blocks.
Returns:
    list: A list of lists where each inner list contains a time block and a random priority value between 0 and 3.
"""


def fixed_pr(bloques_horario, value):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, value])
    return pr_array

"""
Generates a fixed priority array for the given time blocks with the specified priority value.
Args:
    bloques_horario (list): A list of time blocks.
    value (int): The fixed priority value to be assigned to each time block.
Returns:
    list: A list of lists where each inner list contains a time block and the specified priority value.
"""


def update_no_disp_mat(materia, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in materia.no_disponible:
            materia.no_disponible.append(bloques_horario[i])
"""
Updates the 'no_disponible' list of a subject by adding the specified time blocks.
Args:
    materia (Materia): The subject whose availability is being updated.
    bloques_horario (list): A list of time blocks.
    no_disp_index (list): A list of indices indicating which time blocks the subject is not available for.
"""


def update_prioridad_mat(materia, bloques_horario, array_prioridad):
    """
    Updates the priority list of a subject based on the given priority array.
    Args:
        materia (Materia): The subject whose priorities are being updated.
        bloques_horario (list): A list of time blocks.
        array_prioridad (list): A list of tuples where each tuple contains a time block index and a priority value.
    """
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp_mat(materia, bloques_horario, [b_id])
        
        prior = Prioridad(value, bloques_horario[b_id], materia = materia)
        if not (prior in materia.prioridades):
            materia.prioridades.append(prior)
    


In [111]:
#superposicion
"""
Calculate the overlap between two given objects.

This function checks if two objects, `m1` and `m2`, are the same or if they have any common groups.
It returns a `Superposicion` object indicating the type of overlap.

Parameters:
m1 (object): The first object to compare. It should have an attribute `grupos` which is a list of groups.
m2 (object): The second object to compare. It should have an attribute `grupos` which is a list of groups.

Returns:
Superposicion: An object representing the overlap. If `m1` and `m2` are the same, it returns `Superposicion(0, m1, m2)`.
               If they share any common groups, it returns `Superposicion(1, m1, m2)`. Otherwise, it returns `Superposicion(0, m1, m2)`.
"""

def calcular_super(m1, m2):
    if m1 == m2:
        return Superposicion(0, m1, m2)
    else:
        s = False
        for g1 in m1.grupos:
            if g1 in m2.grupos:
                s = True
                break
        return Superposicion(1 if s else 0, m1, m2)
    
def fix_super(value, superposicion, mats1, mats2):
    for m1 in mats1:
        for m2 in mats2:
            if m1 != m2:
                superposicion[(m1.id, m2.id)] = Superposicion(value, m1, m2)
                superposicion[(m2.id, m1.id)] = Superposicion(value, m2, m1)
    

## Imprimir

In [112]:
#imprimir
def print_timetable(dias, horarios, u_dict, w_dict, grupos, anios):
    """
    Prints the timetable for given days, schedules, groups, and years.

    Args:
        dias (list): List of days to be included in the timetable.
        horarios (list): List of schedules to be included in the timetable.
        u_dict (dict): Dictionary containing information about subjects.
        w_dict (dict): Dictionary containing information about professors.
        grupos (list): List of groups to be included in the timetable.
        anios (list): List of years to be included in the timetable.

    """
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            print('\t', *[str(d) for d in dias], sep='\t\t')
            print('··········································································································')
            for h in horarios_turno(horarios, g.turno):
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for ms in mats:
                    if len(ms) == 0:
                        lista.append("---")
                    else:
                        # mostrar profesor:
                        # ps = search_profesor(m, w_dict)
                        # lista.append([str(m) + " " + str(*[str(p) for p in search_profesor(m, w_dict)]) for m in ms])
                        # lista.append("/".join(map(str, [str(m) + " " + str(*[str(p) for p in search_profesor(m, w_dict)]) for m in ms])))
                        lista.append("/".join(map(str, ms)))
                        
                print(str(h), *lista, sep='\t\t')

def print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias):
    """
    Prints the timetable for each professor.

    Parameters:
    dias (list): List of days to include in the timetable.
    horarios (list): List of time slots to include in the timetable.
    u_dict (dict): Dictionary containing university-related data.
    w_dict (dict): Dictionary containing week-related data.
    profesores (list): List of professor objects.
    materias (list): List of subject objects.
    """
    for p in profesores:
        if len(intersection([m.nombre for m in materias], p.materias())) == 0:
            continue
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict, materias) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            if len([l for l in lista if l != "---"]) > 0:
                print(str(h), *lista, sep='\t')
            

def intersection(array1, array2):
    ret = []
    for i in array1:
        if i in array2:
            ret.append(i)
    return ret

def print_prioridades(dias, horarios, profesores):
    for p in profesores:
        print('\n', "Profesor:", str(p), p.nombre_completo)
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            prioridades = [str(search_prioridad(BloqueHorario(d,h),p)) for d in dias]
            print(str(h), *prioridades, sep='\t')

def print_prioridades_materia(dias, horarios, materia):
    """
    Prints the priorities for a given subject (materia) based on the provided days and schedules.

    Args:
        dias (list): A list of days.
        horarios (list): A list of schedules corresponding to the days.
        materia (object): An object representing the subject, which should have a 'profesores' attribute.

    """
    print("Materia: ", str(materia))    
    print_prioridades(dias, horarios, materia.profesores)

def search_materia(b, g, u_dict):
    """
    Searches for a 'materia' object in the given dictionary of 'u' objects that matches the specified criteria.

    Args:
        b (str): The schedule (horario) to match.
        g (str): The group (grupo) to match within the 'materia'.
        u_dict (dict): A dictionary where keys are 'u' object IDs and values are 'u' objects.

    Returns:
        materia: The 'materia' object that matches the specified schedule and group, and has a variable 'x' rounded to 1.
        None: If no matching 'materia' object is found.
    """
    ms = []
    for u_id in u_dict:
        u_obj = u_dict[u_id]
        if u_obj.horario == b and g in u_obj.materia.grupos and round(u_obj.variable.x) == 1:
            ms.append(u_obj.materia)
    return ms

def search_profesor(materia, w_dict):
    """
    Searches for professors assigned to a given subject based on a weight dictionary.
    Args:
        materia (object): The subject object which contains a list of professors.
        w_dict (dict): A dictionary with keys as tuples of (subject_id, professor_id) and values as objects 
                       containing a variable attribute with an x property.
    Returns:
        list: A list of professors assigned to the given subject.
    """
    
    if materia is None:
        return []
    
    ps = []
    for p in materia.profesores:
        if round(w_dict[materia.id, p.id].variable.x) == 1:
            ps.append(p)
    return ps

def search_profesor_by_nombre(profesores, nombre):
    for p in profesores:
        if p.nombre == nombre:
            return p
    return None

def search_materias_by_nombre(materias, nombre):
    ret = []
    for m in materias:
        if m.nombre == nombre:
            ret.append(m)
    return ret
    
def search_materia_prof(b, p, u_dict, w_dict, materias):
    """
    Searches for a 'materia' (subject) that matches the given block and professor.
    Args:
        b: The block to search within.
        p: The professor to search for.
        u_dict (dict): A dictionary where keys are 'materia' IDs and values are objects containing scheduling information.
        w_dict (dict): A dictionary where keys are tuples of 'materia' and professor IDs, and values are objects containing assignment information.
        materias (list): A list of 'materia' objects to search through.
    Returns:
        The 'materia' object that matches the given block and professor, or None if no match is found.
    """
    for m in materias:
        if round(w_dict[m.id, p.id].variable.x * u_dict[m.id, b.id()].variable.x) == 1:
            return m
    #     ps = search_profesor(u_obj.materia, w_dict)
    #     if bloque == b and p in ps and round(u_obj.variable.x) == 1:
    #         return u_obj.materia

    # for u_id in u_dict:
    #     u_obj = u_dict[u_id]
    #     bloque = u_obj.horario
    #     ps = search_profesor(u_obj.materia, w_dict)
    #     if bloque == b and p in ps and round(u_obj.variable.x) == 1:
    #         return u_obj.materia
    return None

def search_prioridad(b, p):
    if b in p.no_disponible:
        return "-"
    for pr in p.prioridades:
        if pr.bloque_horario == b:
            return pr.value
        
def grupos_anio(grupos, anio):
    """
    Filters a list of groups by a specific year.

    Args:
        grupos (list): A list of group objects, where each object has an 'anio' attribute.
        anio (int): The year to filter the groups by.

    Returns:
        list: A list of group objects that match the specified year.
    """
    gs = []
    for g in grupos:
        if g.anio == anio:
            gs.append(g)
    return gs

def horarios_turno(horarios, turno):
    """
    Filters a list of schedule objects based on a specified shift.

    Args:
        horarios (list): A list of schedule objects. Each object should have a 'turnos' attribute.
        turno (str): The shift to filter by.

    Returns:
        list: A list of schedule objects that include the specified shift.
    """
    hs = []
    for h in horarios:
        if turno in h.turnos:
            hs.append(h)
    return hs

def horarios_ids_turno(horarios, turno):
    hs = []
    for h in horarios:
        if turno in h.turnos:
            hs.append(h.id)
    return hs



In [113]:
# imprimir para mostrar en Excel
def print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, anios, show_profs=True):
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            # print('\t', *[str(d) for d in dias], sep='\t\t')
            print('', *[str(d) for d in dias], sep='\t')
            for h in horarios_turno(horarios, g.turno):
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for ms in mats:
                    if len(ms) == 0:
                        lista.append("---")
                    else:
                        # mostrar profesor:
                        # ps = search_profesor(m, w_dict)
                        # lista.append([str(m) + " " + str(*[str(p) for p in search_profesor(m, w_dict)]) for m in ms])
                        # print_list = [str(m) + str([str(p) for p in search_profesor(m, w_dict)]) for m in ms]
                        if show_profs:
                            print_list = [str(m) + "[" + ",".join(map(str, [str(p) for p in search_profesor(m, w_dict)])) + "]" for m in ms]
                        else:
                            print_list = [str(m) for m in ms]
                        lista.append("/".join(map(str, print_list)))
                        # lista.append(print_list)
                        # lista.append("/".join(map(str, ms)))
                        
                # print(str(h), *lista, sep='\t\t')
                print(str(h), *lista, sep='\t')


def print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias):
    for p in profesores:
        if len(intersection([m.nombre for m in materias], p.materias())) == 0:
            continue
        print('\n', "Profesor: ", str(p), p.nombre_completo)
        print('', *[str(d) for d in dias], sep='\t')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict, materias) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            if len([l for l in lista if l != "---"]) > 0:
                print(str(h), *lista, sep='\t')

def print_timetable_salones(dias, horarios, niveles):
    print('\t', *[str(d) for d in dias], sep='\t')
    print('··········································································································')
    for h in horarios:
        lista = [str(niveles[d.id, h.id])+"%" for d in dias]
        print(str(h), *lista, sep='\t')




In [114]:
#copiar resultados de variables a hoja de excel

def copy_variables_excel(u_dict, w_dict):

    data_u = {
        'materia': [],
        'dia': [],
        'horario': [],
        'U': []
    }
    for u_i in u_dict:
        data_u['materia'].append(u_dict[u_i].materia.id)
        data_u['dia'].append(u_dict[u_i].horario.dia.id)
        data_u['horario'].append(u_dict[u_i].horario.horario.id)
        data_u['U'].append(round(u_dict[u_i].variable.x))

    data_w = {
        'materia': [],
        'profesor': [],
        'W': []
    }
    for w_i in w_dict:
        data_w['materia'].append(w_dict[w_i].materia.id)
        data_w['profesor'].append(w_dict[w_i].profesor.id)
        data_w['W'].append(round(w_dict[w_i].variable.x))

    with pd.ExcelWriter(output) as writer:
        pd.DataFrame(data_u).to_excel(writer, sheet_name="u", index=False)
        pd.DataFrame(data_w).to_excel(writer, sheet_name="w", index=False)

    

## Buscar / filtrar

In [115]:
#buscar materias
"""
This module provides functions to filter and group subjects (materias) based on professors and groups.
Functions:
    materias_profesor(profesor, materias_total):
        Filters and returns a list of subjects taught by a specific professor.
    agrupar_materias(lista_materias):
        Groups subjects by their string representation and returns a dictionary where keys are subject names and values are lists of subjects.
    materias_grupo(grupo, materias_total):
        Filters and returns a list of subjects that belong to a specific group.
    materias_grupo_ids(grupo, materias_total):
        Filters and returns a list of subject IDs that belong to a specific group.
"""

def materias_profesor(profesor, materias_total):
    mats = []

    for m in materias_total:
        if profesor in m.profesores:
            mats.append(m)

    return mats

def agrupar_materias(lista_materias):
    lista_nombres = {}

    for m in lista_materias:
        if str(m) not in lista_nombres:
            lista_nombres[str(m)] = [m]
        else:
            lista_nombres[str(m)].append(m)

    return lista_nombres

def materias_grupo(grupo, materias_total):
    materias = []

    for m in materias_total:
        if grupo is not None and grupo in m.grupos:
            materias.append(m)
            
    return materias

def materias_grupo_ids(grupo, materias_total):
    materias_ids = []

    for m in materias_grupo(grupo, materias_total):
        if grupo is not None and grupo in m.grupos:
            materias_ids.append(m.id)
            
    return materias_ids

def electivas(materias):
    return [m for m in materias if m.electiva]

def bloques_horario_materia(materia, bloques_horario):
    ret = []
    for b_id in bloques_horario:
        if set(materia.turnos()).issubset(set(bloques_horario[b_id].horario.turnos)):
            ret.append(b_id)
    return ret



# Formulacion

## (1) Datos / Constantes

### Horarios

In [116]:
"""
    This script initializes and processes scheduling data for a university timetable.

    Classes:
        Dia: Represents a day of the week with an ID and name.
        Horario: Represents a time slot with an ID, start time, end time, and a list of periods.
        BloqueHorario: Represents a block of time on a specific day.

    Variables:
        dias (list of Dia): A list of days of the week.
        dias_ids (list of int): A list of IDs corresponding to the days of the week.
        horarios (list of Horario): A list of time slots.
        horarios_ids (list of int): A list of IDs corresponding to the time slots.
        bloques_horario (dict of tuple: BloqueHorario): A dictionary mapping (day ID, time slot ID) to BloqueHorario objects.
        bloques_horario_ids (list of tuple): A list of tuples representing the keys of the bloques_horario dictionary.
    """

dias = [
    Dia(1, "lun"),
    Dia(2, "mar"),
    Dia(3, "mie"),
    Dia(4, "jue"),
    Dia(5, "vie")
]
dias_ids = []
for d in dias:
    dias_ids.append(d.id)

DIAS = dias

In [117]:
horarios = []

df = pd.read_excel(input, sheet_name="turnos")

for i in range(len(df)):
    id = df["#"][i]
    inicio = df["Inicio"][i]
    fin = df["Fin"][i]

    turnos = []
    t = 1
    while True:
        try:
            new_t = df["Turno " + str(t)][i]
            if isinstance(new_t, str):
                turnos.append(new_t)
            t += 1
        except:
            break

    try:
        id = int(id)
        horarios.append(Horario(id, inicio.strftime("%H:%M"), fin.strftime("%H:%M"), turnos))
    except:
        pass


# horarios = [
#     Horario(1, "8:00", "8:50", ["m"]),
#     Horario(2, "8:50", "9:40", ["m"]),
#     Horario(3, "9:50", "10:40", ["m"]),
#     Horario(4, "10:40", "11:30", ["m"]),
#     Horario(5, "11:40", "12:30", ["m"]),
#     Horario(6, "12:30", "13:20", ["m"]),
#     Horario(7, "14:00", "14:50", ["t"]),
#     Horario(8, "14:50", "15:40", ["t"]),
#     Horario(9, "15:50", "16:40", ["t"]),
#     Horario(10, "16:40", "17:30", ["t"])
# ]

horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)

HORARIOS = horarios

bloques_horario = {}
for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)

BLOQUES_HORARIO = bloques_horario


### Profesores

In [118]:
profesores = []

df = pd.read_excel(input, sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]
    nombre_completo = df["Nombre"][i]
    id = df["ID"][i]

    # # grupos_max = df["Max grupos"][i]
    # if not isinstance(grupos_max, int):
    #     grupos_max = None
    min_max_dias = df["Min/Max dias"][i]
    if min_max_dias not in ["min", "max"]:
        min_max_dias = None

    # if str(cantidad_dias) == "nan":
    #     cantidad_dias = None
    # else:
    #     cantidad_dias = int(cantidad_dias)
        
    if isinstance(nombre, str):
        add_profesor(profesores, int(id), nombre, min_max_dias, nombre_completo)

profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)

PROFESORES = profesores


c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [119]:
print(*[p for p in profesores], sep=", ")

aca, ad, adc, adr, am, ame, an, anp, ap, apa, apr, ar, arqap, as, cm, cp, cr, db, dc, dd, df, dj, dl, dm, ds, ep, epe, er, es, fb, fr, gaa, gb, gba, gc, gca, gd, gdo, gg, gp, gq, guc, hf, hr, ht, icc1, icc2, icc3, jcc, jd, jds, jf, jg, jj, jk, jl, jm, jmi, jpa, jpe, jpf, jpi, lc, lg, ma, mb, mbo, mc, md, mf, mg, mjc, mk, mp, mpi, mpr, mr, mre, ms, mz, nd, nn, ns, ok, p1a, p1b, p1c, pn, po, pp, ps, pu, rc, rl, rr, rs, sb, sgp, sp, svi, svv, th, vt, xm


### Prioridades

In [120]:
# prioridades fijas

# for p in profesores:
#     update_prioridad(p, bloques_horario, fixed_pr(bloques_horario, 1)) # prioridad = 1 para todos los casos
    

In [121]:
# prioridades desde excel: 1 profesor por hoja
"""
Processes scheduling priorities for professors from an Excel file.
For each professor in the `profesores` list, this function attempts to read their
scheduling priorities from a specified Excel file. If the sheet corresponding to
the professor does not exist, it assigns random priorities. Otherwise, it extracts
the scheduling priorities from the Excel sheet and updates the professor's priorities.
Steps:
1. Attempt to read the Excel sheet corresponding to the professor.
2. If the sheet does not exist, assign random priorities and continue to the next professor.
3. If the sheet exists, extract the scheduling priorities.
4. Update the professor's scheduling priorities with the extracted data.
Parameters:
- profesores (list): List of professor identifiers.
- bloques_horario (list): List of available time blocks.
- pd (module): Pandas module for reading Excel files.
- update_prioridad (function): Function to update the professor's scheduling priorities.
- random_pr (function): Function to generate random priorities for a professor.
Excel File Structure:
- Each sheet corresponds to a professor.
- The first column contains time block identifiers.
- Subsequent columns contain priority values for each time block.
Example:
    profesores = ['ProfA', 'ProfB']
    bloques_horario = ['Block1', 'Block2']
    update_prioridad = some_update_function
    random_pr = some_random_function
    # Call the function with the above parameters
"""


for prof in profesores:

    try:
        df = pd.read_excel(input, sheet_name=str(prof))
    except:
        # update_prioridad(prof, bloques_horario, random_pr(bloques_horario))
        update_prioridad(prof, bloques_horario, fixed_pr(bloques_horario, 1)) # prioridad = 1 para todos los casos
        continue

    array_prioridad = []
    # array_prioridad[i] = [(d,h),a]

    suma_prioridades = 0

    h_id = []

    hids = True
    for column in df:
        if hids:
            for row in df[column]:
                h_id.append(row)
            hids = False
        else:
            i = 0
            for row in df[column]:
                array_prioridad.append([(column, h_id[i]), row])
                suma_prioridades += row
                i += 1
    # print(array_prioridad)
    
    if suma_prioridades == 0:
        update_prioridad(prof, bloques_horario, fixed_pr(bloques_horario, 1))
        continue

    update_prioridad(prof, bloques_horario, array_prioridad)


### Grupos

In [122]:
#grupos
"""
This script processes an Excel file containing group information and populates a list of group objects.
Functions:
    add_grupo(grupos, *args): Adds a group to the list of groups.
Variables:
    grupos (list): A list to store group objects.
    df (DataFrame): DataFrame containing the data read from the Excel file.
    grupos_ids (list): A list to store the IDs of the groups.
Workflow:
1. Read the Excel file input from the sheet named "grupos" into a DataFrame.
2. Iterate over each row in the DataFrame and extract relevant information.
3. Append extracted information to a list of arguments.
4. Replace 'nan' values with None and attempt to convert arguments to integers.
5. Call the add_grupo function with the list of arguments to add a group to the grupos list.
6. Populate the grupos_ids list with the IDs of the groups.
"""
grupos = []


df = pd.read_excel(input, sheet_name="grupos")

for i in range(len(df)):
    
    args = []
    args.append(df["Año"][i])  # anio
    args.append(df["Turno"][i])     # turno
    args.append(df["Carrera"][i])   # carrera
    args.append(df["Particion"][i])  # particion
    args.append(isinstance(df["Recurse"][i], str)) # recurse
    args.append(isinstance(df["Auxiliar"][i], str))

    for i in range(0, len(args)):
        
        if str(args[i]) == "nan":
            args[i] = None

        try:
            args[i] = int(args[i])
        except:
            pass      

    add_grupo(grupos, *args)

grupos_ids = []
for g in grupos:
    grupos_ids.append(g.id)

GRUPOS = grupos

c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [123]:
print(*[str(g) for g in grupos])

11 12 13 14 1LIC 1REC1 1REC2 2CIV 2IND 2INF1 2INF2 2TEL 2LIC 3CIV 3IND 3TEL 3INF1 3INF2 3LIC 4CIV 4IND 4TEL 4INF1 4INF2 4LIC 5CIV 5IND 5TEL 5INF


In [124]:
anios = []
"""
This script processes a list of groups and extracts unique years from them.

Variables:
    anios (list): A list to store unique years extracted from the groups.
    grupos (list): A list of group objects, each containing an 'anio' attribute.

The script iterates through each group in 'grupos', checks if the 'anio' attribute is not None and not already in the 'anios' list, and appends it to 'anios' if both conditions are met.
"""
for g in grupos:
    if not g.anio is None and not g.anio in anios:
            anios.append(g.anio)

ANIOS = anios


### Materias 

In [125]:
#materias
"""
This script processes an Excel file containing information about various subjects (materias) and extracts relevant details to create a list of subjects.
The script performs the following steps:
1. Reads the Excel file input from the sheet named "materias".
2. Iterates through each row of the DataFrame to extract subject details.
3. For each subject, it extracts the name, hourly load, number of days per week, and the number of professors.
4. It then attempts to extract the names of the professors and groups associated with each subject.
5. Finally, it adds the subject to the list of subjects using the `add_materia` function.
Variables:
- materias: A list to store the details of each subject.
- df: DataFrame containing the data read from the Excel file.
- nombre: The name of the subject.
- carga_horaria: The hourly load of the subject.
- cantidad_dias: The number of days per week the subject is taught.
- cantidad_profesores: The number of professors teaching the subject.
- profs: A list to store the names of the professors.
- gs: A list to store the names of the groups.
Functions:
- add_materia: Adds a subject to the list of subjects.
- lista_grupos: Processes and returns a list of groups.
- lista_profesores: Processes and returns a list of professors.
"""

materias = []

df = pd.read_excel(input, sheet_name="materias")

for i in range(len(df)):

    nombre = df["Materia"][i]
    if not isinstance(nombre, str):
        continue # cancel if no name is provided

    id = df["ID"][i]

    carga_horaria = df["Carga Horaria"][i]
    cantidad_dias = df["Días por semana"][i]
    electiva = isinstance(df["Electiva"][i], str)
    teo_prac = df["Teo/Prac"][i]
    if teo_prac not in ["teo", "prac"]:
        teo_prac = None
    
    try:
        cantidad_profesores = int(df["Cantidad profesores"][i])
    except:
        cantidad_profesores = 1 # default value

    profs = []
    p = 1
    while True:
        try:
            new_p = df["Profesor " + str(p)][i]
            if isinstance(new_p, str):
                profs.append(new_p)
            p += 1
        except:
            break

    gs = []
    g = 1
    while True:
        try:
            new_g = df["Grupo " + str(g)][i]
            if isinstance(new_g, str):
                gs.append(new_g)
            g += 1
        except:
            break

    
    
    # gs = lista_grupos(grupos, [x.strip() for x in str(df["Grupos"][i]).split(',')])
    # profs = lista_profesores(profesores, [x.strip() for x in str(df["Profesores"][i]).split(',')])

    add_materia(materias, int(id), nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs), cantidad_profesores, electiva, teo_prac)


materias_ids = []
# materias_dict = {}
for m in materias:
    materias_ids.append(m.id)
    # materias_dict[m.id] = m

MATERIAS = materias


c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [126]:
# materias por profesor:
"""
This script processes an Excel file containing information about professors and their respective subjects.
It reads the data from the specified sheet and updates the list of subjects for each professor.
Steps:
1. Load the Excel file "input.xlsm" and read the sheet named "profesores" into a DataFrame.
2. Iterate through each row of the DataFrame to extract professor names and their subjects.
3. For each professor, gather the subjects they teach along with the maximum number of groups for each subject.
4. Update the professor's list of subjects using the `search_profesor_by_nombre` function.
Variables:
- df: DataFrame containing the data from the Excel sheet.
- nombre: Name of the professor.
- lista_mats: List of dictionaries, each containing the subject name and the maximum number of groups.
- m: Counter for the subject columns in the DataFrame.
- new_m: Name of the subject being processed.
- grupos_max: Maximum number of groups for the subject.
- prof: Professor object found using the `search_profesor_by_nombre` function.
Functions:
- search_profesor_by_nombre(profesores, nombre): Searches for a professor by name in the given list of professors.
Note:
- The script assumes that the Excel file and sheet structure are consistent with the expected format.
- The `search_profesor_by_nombre` function and other commented-out functions need to be defined elsewhere in the codebase.
"""
df = pd.read_excel(input, sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]

    lista_mats = []
    m = 1
    while True:
        try:
            new_m = df["Materia " + str(m)][i]
            
            if isinstance(new_m, str):
                grupos_max = int(df["Grupos materia " + str(m)][i])
                lista_mats.append({"nombre_materia" : new_m, "grupos_max" : grupos_max})
            m += 1
        except:
            break
    
    prof = search_profesor_by_nombre(profesores, nombre)
    if not prof is None:
        prof.lista_materias = lista_mats


    # add_materia(materias, id, nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs))


c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [127]:
# for p in profesores:
#     mats = ""
#     for m in materias:
#         if p in m.profesores:
#             mats += str(m) + " "
#     print(p, ":", mats)



In [128]:
# print(*[[p.lista_materias, p.nombre] for p in profesores], sep="\n")

In [129]:
# for m in materias:
#     print_prioridades_materia(dias, horarios, m)

### Superposicion

In [130]:
# superposicion
"""
Calculates the overlap between pairs of subjects and stores the results in a dictionary.

This script iterates over all pairs of subjects in the `materias` list and calculates the overlap 
between each pair using the `calcular_super` function. The results are stored in the `superposicion` 
dictionary, where the keys are tuples of subject IDs and the values are the calculated overlaps.

Variables:
    superposicion (dict): A dictionary to store the overlap results between pairs of subjects.
    materias (list): A list of subject objects, each with an `id` attribute.

Functions:
    calcular_super(m1, m2): A function that takes two subject objects and returns the overlap between them.

Dictionary Structure:
    superposicion[(m1.id, m2.id)] = overlap_value
"""
superposicion = {}

for m1 in materias:
    for m2 in materias:
        superposicion[(m1.id, m2.id)] = calcular_super(m1, m2)



In [131]:

# CASO PARTICULAR: permitir superposicion de Estadistica Aplicada con Metodologia de la Investigacion
fix_super(0, superposicion,
            [m for m in materias if m.nombre == "EA"],
            [m for m in materias if m.nombre == "MInv"]
            )


### Electivas

Fijar superposicion = 0 (permitir superposicion)

In [132]:
for e in electivas(materias):
    fix_super(0, superposicion, [e], electivas(materias))

SUPERPOSICION = superposicion

In [133]:
superposicion_electivas = {}

for m1 in electivas(materias):
    for m2 in electivas(materias):
        superposicion_electivas[(m1.id, m2.id)] = calcular_super(m1, m2)

In [134]:

# AgroN con todas las demas electivas
fix_super(0, superposicion_electivas, search_materias_by_nombre(electivas(materias), "AgroN"), electivas(materias))

# StartUps 1 con todas las demas electivas
fix_super(0, superposicion_electivas, search_materias_by_nombre(electivas(materias), "StUp1"), electivas(materias))

# BPMN y dbdIII coincidan horarios
# fix_super(0, superposicion, search_materias_by_nombre(electivas(materias), "BPMN"), search_materias_by_nombre(electivas(materias), "DBD3"))

# # RPA y aplicaciones móviles coincidan horarios
# fix_super(0, superposicion, search_materias_by_nombre(electivas(materias), "RPA"), search_materias_by_nombre(electivas(materias), "ApMov"))

# # Diseño estructural y Pavimentos coincidan horarios
# fix_super(0, search_materias_by_nombre(electivas(materias), "DisEst"), search_materias_by_nombre(electivas(materias), "DPav"))

# Diseño estructural + Puentes pueden coincidir con Diseño de Pavimentos + Diseño de infraestructura pluvial y urbana
# fix_super(0, superposicion,
#             [m for m in electivas(materias) if m.nombre in ["DisEst", "Puentes"]],
#             [m for m in electivas(materias) if m.nombre in ["DPav", "DIPyU"]]
#             )

SUPERPOSICION_ELECTIVAS = superposicion_electivas

In [135]:
print("Grupos:", len(grupos))
# print([str(i) for i in grupos])
print("Materias:", len(materias))
# print([str(i) for i in materias])
print("Profesores:", len(profesores))
# print([str(i) for i in profesores])

Grupos: 29
Materias: 167
Profesores: 104


### Asignaciones previas

In [136]:
U_prev = {}
df = pd.read_excel(output, sheet_name="u")

for i in range(len(df)):
    
    materia = df["materia"][i]
    dia = df["dia"][i]
    horario = df["horario"][i]
    valor = df["U"][i]
        
    U_prev[materia, (dia, horario)] = valor

In [137]:
W_prev = {}
df = pd.read_excel(output, sheet_name="w")

for i in range(len(df)):
    
    materia = df["materia"][i]
    profesor = df["profesor"][i]
    valor = df["W"][i]
        
    W_prev[materia, profesor] = valor


In [138]:
Y_prev = {}

for p in profesores:
    for b_id in bloques_horario:
        sum = np.sum([U_prev[m.id, b_id] * W_prev[m.id, p.id] for m in materias])
        Y_prev[p.id, b_id] = min(1, sum) # 'or' en lugar de suma
        # if sum > 1:
        #     print(sum, Y_prev[p.id, b_id], p, bloques_horario[b_id])


## (2) Variables


$$ u_{mb} \in \{0,1\} $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} \in \{0,1\} $$

    m: materia
    d: dia

$$ w_{mp} \in \{0,1\} $$

    m: materia
    p: profesor


$$ x_{gb} \in \{0,1\} $$

    g: grupo
    b: bloque horario = [dh]: dia y hora

$$ y_{pb} \in \{0,1\} $$

    p: profesor
    b: bloque horario = [dh]: dia y hora

$$ z_{pd} \in \{0,1\} $$

    p: profesor
    d: dia

In [139]:
# variables
"""
This script initializes and populates several dictionaries to map combinations of entities 
(materias, bloques_horario, dias, profesores, and grupos) to corresponding function outputs.
Variables:
    u_dict (dict): Maps tuples of (materia.id, bloque_horario) to the result of function u(m, bloques_horario[b_id]).
    v_dict (dict): Maps tuples of (materia.id, dia.id) to the result of function v(m, d).
    w_dict (dict): Maps tuples of (materia.id, profesor.id) to the result of function w(m, p).
    x_dict (dict): Maps tuples of (grupo.id, bloque_horario) to the result of function x(g, bloques_horario[b_id]).
    y_dict (dict): Maps tuples of (profesor.id, bloque_horario) to the result of function y(p, bloques_horario[b_id]).
    z_dict (dict): Maps tuples of (profesor.id, dia.id) to the result of function z(p, d).
Prints:
    The length of each dictionary after it has been populated.
"""
def initialize_variables(materias, bloques_horario, dias, profesores, grupos):
    u_dict = {}
    for m in materias:
        for b_id in bloques_horario:
            u_dict[(m.id, b_id)] = u(m, bloques_horario[b_id])
    print("u: ", len(u_dict))

    v_dict = {}
    for m in materias:
        for d in dias:
            v_dict[(m.id, d.id)] = v(m, d)
    print("v: ", len(v_dict))

    w_dict = {}
    for m in materias:
        for p in profesores:
            w_dict[(m.id, p.id)] = w(m, p)
    print("w: ", len(w_dict))

    x_dict = {}
    for g in grupos:
        for b_id in bloques_horario:
            x_dict[(g.id, b_id)] = x(g, bloques_horario[b_id])
    print("x: ", len(x_dict))

    y_dict = {}
    for p in profesores:
        for b_id in bloques_horario:
            y_dict[(p.id, b_id)] = y(p, bloques_horario[b_id])
    print("y: ", len(y_dict))

    z_dict = {}
    for p in profesores:
        for d in dias:
            z_dict[(p.id, d.id)] = z(p, d)
    print("z: ", len(z_dict))

    return u_dict, v_dict, w_dict, x_dict, y_dict, z_dict

In [140]:
print(len(materias))
print(len(bloques_horario))

167
75


In [141]:
# Create variables
"""
This script creates binary decision variables for multiple dictionaries using the Gurobi optimization model.
Variables:
    u_dict (dict): Dictionary containing elements for which binary variables "u" are created.
    v_dict (dict): Dictionary containing elements for which binary variables "v" are created.
    w_dict (dict): Dictionary containing elements for which binary variables "w" are created.
    x_dict (dict): Dictionary containing elements for which binary variables "x" are created.
    y_dict (dict): Dictionary containing elements for which binary variables "y" are created.
    z_dict (dict): Dictionary containing elements for which binary variables "z" are created.
Each element in the dictionaries is assigned a binary variable using the Gurobi model's `addVar` method.
"""

# u_vars = m.addMVar(shape=len(u_dict), vtype=GRB.BINARY, name="u") # variable matrix

def create_variables(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict):
    
    for u_i in u_dict: # crear variables "u" a partir de u_dict
        u_dict[u_i].variable = model.addVar(vtype=GRB.BINARY, name=str(u_dict[u_i]))

    for v_i in v_dict: # crear variables "v" a partir de v_dict
        v_dict[v_i].variable = model.addVar(vtype=GRB.BINARY, name=str(v_dict[v_i]))

    for w_i in w_dict: # crear variables "v" a partir de v_dict
        w_dict[w_i].variable = model.addVar(vtype=GRB.BINARY, name=str(w_dict[w_i]))


    for x_i in x_dict: # crear variables "x" a partir de x_dict
        x_dict[x_i].variable = model.addVar(vtype=GRB.BINARY, name=str(x_dict[x_i]))

    for y_i in y_dict: # crear variables "y" a partir de y_dict
        y_dict[y_i].variable = model.addVar(vtype=GRB.BINARY, name=str(y_dict[y_i]))

    for z_i in z_dict: # crear variables "z" a partir de z_dict
        z_dict[z_i].variable = model.addVar(vtype=GRB.BINARY, name=str(z_dict[z_i]))


## (3) Restricciones

### (3.1) Materias

#### (3.1.1) superposicion (redundante con la definicion de la variable x)
$$ \sum_{b} \sum_{m, m'}{u_{mb} \times S_{m,m'} \times u_{m'b}} = 0 $$


In [142]:
"""
    This code defines constraints to ensure that no two subjects (materias) overlap
    in the same time block (bloque horario).
    Constraints:
        1. For each time block (b) in bloques_horario_ids, the sum of the product of the decision variables 
        (u_dict[m1, b].variable and u_dict[m2, b].variable) and the overlap value (superposicion[(m1, m2)].value)
        for all pairs of subjects (m1, m2) must be equal to 0.
"""

def constr_superposicion(model, u_dict):

    model.addConstrs(gp.quicksum(u_dict[m1, b].variable * superposicion[(m1, m2)].value * u_dict[m2, b].variable
                            for m1 in materias_ids for m2 in materias_ids)
                == 0 for b in bloques_horario_ids)


## for g in grupos:
##     mats = materias_grupo(g, materias)
##     if len(mats) > 1:
##         # model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)
##         model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)


In [143]:
# print(*[[str(m), m.cantidad_dias] for m in materias], sep="\n")

#### (3.1.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = C_m $$

    para todo m (c_m: carga horaria)

In [144]:
def constr_carga_horaria(model, u_dict):
    for m in materias:
        model.addConstr(gp.quicksum(u_dict[m.id, b].variable for b in bloques_horario_ids) == m.carga_horaria)

#### (3.1.3) particion de horas por materia

##### (3.1.3.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [145]:
def constr_dias_materia(model, v_dict):
    for m in materias:
        model.addConstr(gp.quicksum(v_dict[m.id, d].variable for d in dias_ids) == m.cantidad_dias)

##### (3.1.3.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times {H_{MIN}}_m \leq \sum_h {u_{mdh}} \leq v_{md} \times {H_{MAX}}_m $$

    para todo m, d

In [146]:
def constr_max_min_horas(model, u_dict, v_dict):

    for m in materias:

        model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                        <= m.horas_max() * v_dict[m.id,d].variable for d in dias_ids)
        
        model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                        >= m.horas_min() * v_dict[m.id,d].variable for d in dias_ids)
    

##### (3.1.3.3) fijar materia a turno de horarios

$$ u_{mb} = 0 $$

    para todo b fuera de m.turnos
    para todo m



In [147]:
def constr_turnos_materia(model, u_dict):

    for m in materias:
        no_bloques_materia_ids = [b for b in bloques_horario if b not in bloques_horario_materia(m, bloques_horario)]
        # for b in bloques_horario:
        #     bloques_horario_materia(m, bloques_horario)
        #     if len([t for t in bloques_horario[b].horario.turnos if t in m.turnos()]) == 0:
        #         no_bloques_materia_ids.append(b)
        model.addConstrs(u_dict[m.id, b].variable == 0 for b in no_bloques_materia_ids)


#### (3.1.4) horas consecutivas dentro de un dia

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [148]:
def constr_horas_consecutivas(model, u_dict, v_dict):

    model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)

Alternativa:
$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} \leq 1 $$
    para todo m, d

#### (3.1.5) evitar dias consecutivos para una misma materia

$$ \sum_m \sum_d {v_{m(d)} · v_{m(d+1)}} = 0 $$


In [149]:
def constr_dias_consecutivos(model, v_dict):
    model.addConstrs(gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:-1]) == 0
                #  for m in materias_ids)
                 for m in [mat.id for mat in materias if not mat.electiva])


#### (3.1.6) indisponibilidad de materia (opcional)

$$ u_{mb} = 0 $$

    para todo m sin disponibilidad en b

In [150]:
def constr_no_disponible_materia(model, u_dict):
    for m in materias:
        for b in m.no_disponible:
            model.addConstr(u_dict[m.id, b.id()].variable == 0)

### (3.2) Profesores

#### (3.2.1) indisponibilidad

$$ \sum_m w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [151]:
def constr_no_disponible_profesor(model, u_dict, w_dict):
    for m in materias:
        for p in m.profesores:
            for b in p.no_disponible:
                model.addConstr(u_dict[m.id, b.id()].variable * w_dict[m.id, p.id].variable == 0)

LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ y_{pb} = 0 $$

    para todo p sin disponibilidad en b

In [152]:
def constr_no_disponible_profesor_lineal(model, y_dict):
    for p in profesores:
        for b in p.no_disponible:
            model.addConstr(y_dict[p.id, b.id()].variable == 0)

#### (3.2.2) unica materia por profesor para un mismo bloque horario

$$ \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

REDUNDANTE CON DEFINICION DE y_pb

In [153]:
def constr_unica_materia_profesor(model, u_dict, w_dict):

    p_grupos_simultaneos = ["rs", "er", "dd", "ns", "jl", "fb"]

    for p in [p for p in profesores if p.nombre not in p_grupos_simultaneos]:
        model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 1 for b in bloques_horario_ids)
        
    # caso particular
    for p in [p for p in profesores if p.nombre in p_grupos_simultaneos]:
        model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 2 for b in bloques_horario_ids)

#### (3.2.3) profesores por materia


##### (3.2.3.1) limitar profesores a lista

$$ w_{mp} = 0 $$

    para todo p not in m.lista_profesores
    para todo m

In [154]:
def constr_limitar_profesores_materia(model, w_dict):
    
    for m in materias:
        for p in profesores:
            if p not in m.profesores:
                model.addConstr(w_dict[m.id, p.id].variable == 0)

##### (3.2.3.2) cantidad de profesores para una misma materia

$$ \sum_p {w_{mp}} = P_m $$

    para todo m

In [155]:
def constr_cantidad_profesores(model, w_dict):
    for m in materias:
        model.addConstr(gp.quicksum(w_dict[m.id, p].variable for p in profesores_ids) == m.cantidad_profesores)

#### (3.2.4) carga horaria por docente: limitar la cantidad de materias por profesor


$$ \sum_m {w_{mp}} \leq K_p $$

    para todo p



In [156]:
# separar por nombre de materia
def constr_grupos_max_profesor(model, w_dict):
    for p in profesores:
        for l in p.lista_materias:
            mats = search_materias_by_nombre(materias, l["nombre_materia"])
            if len(mats) > 0:
                model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in mats) <= l["grupos_max"])
        

#### (3.2.5) definicion de variable "y" (asignacion horario-profesor)

$$ y_{pb} = \sum_m{u_{mb}·w_{mp}} $$

    para todo p, b

In [157]:
def constr_definir_y(model, y_dict, u_dict, w_dict):

    uw_vars = {} # variable auxiliar = u_mb·w_mp
    for p in profesores:
        for m in materias_profesor(p, materias):
            # for b_id in bloques_horario_materia(m, bloques_horario):
            for b_id in bloques_horario_ids:
                uw_vars[m.id,b_id,p.id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+str(b_id)+str(p)+"_uw")
                model.addConstr(uw_vars[m.id,b_id,p.id] == u_dict[m.id,b_id].variable * w_dict[m.id,p.id].variable)

        model.addConstrs(y_dict[p.id,b_id].variable ==
                     gp.or_(uw_vars[m_id,b_id,p.id] for m_id in [m.id for m in materias_profesor(p, materias)])
                     for b_id in bloques_horario_ids)
    
    # model.addConstrs(y_dict[p, b].variable ==
    #              gp.quicksum(u_dict[m,b].variable * w_dict[m,p].variable for m in materias_ids)
    #              for b in bloques_horario_ids for p in profesores_ids)

#### (3.2.6) definicion de variable "z" (para dias con clase por profesor)

$$ z_{pd} = OR_h{y_{p(dh)}} $$

    para todo p, d

In [158]:
def constr_definir_z(model, z_dict, y_dict):
    
    model.addConstrs(z_dict[p,d].variable ==
                gp.or_(y_dict[p,(d,h)].variable for h in horarios_ids)
                for p in profesores_ids for d in dias_ids)

Posible linealizacion:

$$ z_{pd} \leq \sum_h {y_{pdh}} \leq M \times z_{pd} $$

    para todo m, d


In [159]:
# for p in profesores:

#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      <= 10 * z_dict[p.id,d].variable for d in dias_ids)
    
#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      >= z_dict[p.id,d].variable for d in dias_ids)

#### (3.2.7) limitar la cantidad de horas diarias por profesor (NO VA)


$$ \sum_{m,h} {w_{mp}·u_{mdh}} \leq {HD}_p $$

    para todo p, d


In [160]:
# FALTA CHEQUEAR

# for p in profesores:
#     if not p.horas_max is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstrs(gp.quicksum(
#             w_dict[m, p.id].variable * u_dict[m,(d,h)].variable for m in materias_ids for h in horarios_ids
#             ) <= p.horas_max for d in dias_ids)

#### (3.2.8) limitar dias con clase por docente (NO VA COMO RESTRICCION)


$$ \sum_{m,d} {w_{mp}·v_{md}} \leq {Dmax}_p $$
$$ \sum_{d} {z_{pd}} \leq {Dmax}_p $$

    para todo p


In [161]:
# MAL IMPLEMENTADA

# for p in profesores:
#     if not p.cantidad_dias is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstr(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ) <= p.cantidad_dias)
        

In [162]:
# p = search_profesor_by_nombre(profesores, "jf")
# print(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ).getValue())

### (3.3) Restricciones externas

#### (3.3.1) cantidad de salones

$$ \sum_{m} {u_{mb}} \leq K $$

    para todo b

In [163]:
K = 13      # 13 salones en este caso

def constr_cantidad_salones(model, u_dict):
    model.addConstrs(gp.quicksum(u_dict[m, b].variable for m in materias_ids) <= K for b in bloques_horario_ids)

#### (3.3.2) restricciones ad hoc

In [164]:
def constr_ad_hoc(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict):

    # MATERIAS

    # no dias consecutivos y no coincidir dia para ADA
    ada = [m.id for m in search_materias_by_nombre(materias, "ADA")]
    if len(ada) == 2:
        model.addConstr(gp.quicksum(v_dict[ada[0],d].variable * v_dict[ada[1],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[ada[1],d].variable * v_dict[ada[0],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[ada[0],d].variable * v_dict[ada[1],d].variable for d in dias_ids) == 0)

    # no dias consecutivos y no coincidir dia para SyMH
    symh = [m.id for m in search_materias_by_nombre(materias, "SyMH")]
    if len(symh) == 2:
        model.addConstr(gp.quicksum(v_dict[symh[0],d].variable * v_dict[symh[1],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[symh[1],d].variable * v_dict[symh[0],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[symh[0],d].variable * v_dict[symh[1],d].variable for d in dias_ids) == 0)

    # no dias consecutivos y no coincidir dia para SD
    sd = [m.id for m in search_materias_by_nombre(materias, "SD")]
    if len(sd) == 2:
        model.addConstr(gp.quicksum(v_dict[sd[0],d].variable * v_dict[sd[1],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[sd[1],d].variable * v_dict[sd[0],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[sd[0],d].variable * v_dict[sd[1],d].variable for d in dias_ids) == 0)

    # no dias consecutivos para QC
    for qc in [m.id for m in search_materias_by_nombre(materias, "QC")]:
        model.addConstr(gp.quicksum(v_dict[qc,d].variable * v_dict[qc,d+1].variable for d in dias_ids[0:-1]) == 0)

    # no coincidir dia para RPA
    rpa = [m.id for m in search_materias_by_nombre(materias, "RPA")]
    if len(rpa) == 2:
        # model.addConstr(gp.quicksum(v_dict[rpa[0],d].variable * v_dict[rpa[1],d+1].variable for d in dias_ids[0:-1]) == 0)
        # model.addConstr(gp.quicksum(v_dict[rpa[1],d].variable * v_dict[rpa[0],d+1].variable for d in dias_ids[0:-1]) == 0)
        model.addConstr(gp.quicksum(v_dict[rpa[0],d].variable * v_dict[rpa[1],d].variable for d in dias_ids) == 0)

    # no coincidir dia para Teologia
    teo = [m.id for m in search_materias_by_nombre(materias, "Teo")]
    if len(teo) != 0 and len(teo) % 2 == 0:
        for i in range(0,len(teo),2):
            model.addConstr(gp.quicksum(v_dict[teo[i],d].variable * v_dict[teo[i+1],d].variable for d in dias_ids) == 0)
        # for i in [2,4]:
        # # no dias consecutivos salvo para 3LIC
        #     model.addConstr(gp.quicksum(v_dict[teo[i],d].variable * v_dict[teo[i+1],d+1].variable for d in dias_ids[0:-1]) == 0)
        #     model.addConstr(gp.quicksum(v_dict[teo[i+1],d].variable * v_dict[teo[i],d+1].variable for d in dias_ids[0:-1]) == 0)

    # no dias consecutivos y no coincidir dia para Economia (grupos 2INF1 y 2TEL/2INF2)
    eco_gs = []
    eco_gs.append([m for m in search_materias_by_nombre(materias, "Eco") if "2INF1" in [str(g) for g in m.grupos]])
    eco_gs.append([m for m in search_materias_by_nombre(materias, "Eco") if "2TEL" in [str(g) for g in m.grupos]])
    for g in eco_gs:
        if len(g) == 2:
            model.addConstr(gp.quicksum(v_dict[g[0].id, d].variable * v_dict[g[1].id, d+1].variable for d in dias_ids[0:-1]) == 0)
            model.addConstr(gp.quicksum(v_dict[g[1].id, d].variable * v_dict[g[0].id, d+1].variable for d in dias_ids[0:-1]) == 0)
            model.addConstr(gp.quicksum(v_dict[g[0].id, d].variable * v_dict[g[1].id, d].variable for d in dias_ids) == 0)

        # fijar al final para bloques de 1 hora (se adiciona horario excepcional)
        for b in g:
            if b.carga_horaria == 1:
                model.addConstr(gp.quicksum(u_dict[b.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "m")[0:-1]) == 0)

    # fijar MDp al ultimo bloque (se adiciona horario excepcional)
    for mdp in search_materias_by_nombre(materias, "MDp"):
        if mdp.carga_horaria == 1:
            model.addConstr(gp.quicksum(u_dict[mdp.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "m")[0:-1]) == 0)

    # fijar a ultimo bloque MecS2p
    for mecs2p in search_materias_by_nombre(materias, "MecS2p"):
        model.addConstr(gp.quicksum(u_dict[mecs2p.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "t2")[0:-1]) == 0)    
    
    # coincidir horarios para ambos grupos de SDig
    sdig = [m.id for m in search_materias_by_nombre(materias, "SDig")]
    if len(sdig) == 2:
        model.addConstrs(u_dict[sdig[0], b].variable == u_dict[sdig[1], b].variable for b in bloques_horario_ids)

    # coincidir horarios para ambos grupos de AM2
    am2 = [m.id for m in search_materias_by_nombre(materias, "AM2") + search_materias_by_nombre(materias, "AM2p")]
    if len(am2) == 4:
        model.addConstrs(u_dict[am2[0], b].variable == u_dict[am2[1], b].variable for b in bloques_horario_ids)
        model.addConstrs(u_dict[am2[2], b].variable == u_dict[am2[3], b].variable for b in bloques_horario_ids)

    # coincidir horarios para ambos grupos de TIC2
    tic2 = [m.id for m in search_materias_by_nombre(materias, "TIC2")]
    if len(tic2) == 2:
        model.addConstrs(u_dict[tic2[0], b].variable == u_dict[tic2[1], b].variable for b in bloques_horario_ids)
            
    # coincidir EA con MInv
    ea = [m.id for m in search_materias_by_nombre(materias, "EA")]
    mi = [m.id for m in search_materias_by_nombre(materias, "MInv")]
    if len(ea) == len(mi):
        for i in range(len(ea)):
            model.addConstrs(u_dict[ea[i], b].variable == u_dict[mi[i], b].variable for b in bloques_horario_ids)

    # coincidir UIUX electiva para 5TEL/INF con obligatoria para 3LIC (se cubre hora excepcional, es decir menor carga horaria para 5to)
    # fijar a ultimo bloque UIUX electiva
    try:
        uiux_elec = [m.id for m in search_materias_by_nombre(electivas(materias), "UIUX")][0]
        model.addConstr(gp.quicksum(u_dict[uiux_elec, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "t2")[0:-1]) == 0)
        try:
            uiux_lic = [m.id for m in search_materias_by_nombre(materias, "UIUX") if "3LIC" in [str(g) for g in m.grupos]][0]
            model.addConstrs(u_dict[uiux_lic, b].variable >= u_dict[uiux_elec, b].variable for b in bloques_horario_ids)
        except:
            pass
    except:
        pass

    # coincidir TecFP para 5IND y 4IND
    tecfp = [m.id for m in search_materias_by_nombre(materias, "TecFP")]
    if len(tecfp) == 2:
        model.addConstrs(u_dict[tecfp[0], b].variable == u_dict[tecfp[1], b].variable for b in bloques_horario_ids)

    # HC martes y viernes
    for hc in [m.id for m in search_materias_by_nombre(materias, "HC")]:
        model.addConstr(v_dict[hc, 2].variable + v_dict[hc, 5].variable == 2)


    # PROFESORES

    # lunes xor viernes para hr
    hr = search_profesor_by_nombre(profesores, "hr")
    if hr is not None:
        model.addConstr(z_dict[hr.id, 1].variable + z_dict[hr.id, 5].variable <= 1)

    # 1 solo dia para jl
    jl = search_profesor_by_nombre(profesores, "jl")
    if jl is not None:
        model.addConstr(gp.quicksum(z_dict[jl.id, d].variable for d in dias_ids) == 1)

    # 3 dias (maximo posible) para ar (separar mañana y tarde)
    ar = search_profesor_by_nombre(profesores, "ar")
    if ar is not None:
        model.addConstr(gp.quicksum(z_dict[ar.id, d].variable for d in dias_ids) >= 3)

    # 2 dias (minimo posible) para dj (coincidir mañana y tarde)
    dj = search_profesor_by_nombre(profesores, "dj")
    if dj is not None:
        model.addConstr(gp.quicksum(z_dict[dj.id, d].variable for d in dias_ids) == 2)

    # maximo 3 dias para jk
    jk = search_profesor_by_nombre(profesores, "jk")
    if jk is not None:
        model.addConstr(gp.quicksum(z_dict[jk.id, d].variable for d in dias_ids) <= 3)

    # 1 dia para jf
    jf = search_profesor_by_nombre(profesores, "jf")
    if jf is not None:
        model.addConstr(gp.quicksum(z_dict[jf.id, d].variable for d in dias_ids) == 1)

    # 2 dias para ds
    ds = search_profesor_by_nombre(profesores, "ds")
    if ds is not None:
        model.addConstr(gp.quicksum(z_dict[ds.id, d].variable for d in dias_ids) == 2)

    # 2 dias para ad
    ad = search_profesor_by_nombre(profesores, "ad")
    if ad is not None:
        model.addConstr(gp.quicksum(z_dict[ad.id, d].variable for d in dias_ids) == 2)

    # evitar 4 horas seguidas para mp
    mp = search_profesor_by_nombre(profesores, "mp")
    if mp is not None:
        model.addConstrs(y_dict[mp.id, (d,h)].variable
                        + y_dict[mp.id, (d,h+1)].variable
                        + y_dict[mp.id, (d,h+2)].variable
                        + y_dict[mp.id, (d,h+3)].variable
                        <= 3
                        for h in horarios_ids[0:-3]
                        for d in dias_ids)
        # alternativa no lineal:
        # model.addConstr(gp.quicksum(
        #                 y_dict[mp.id, (d,h)].variable
        #                 * y_dict[mp.id, (d,h+1)].variable
        #                 * y_dict[mp.id, (d,h+2)].variable
        #                 * y_dict[mp.id, (d,h+3)].variable
        #                 for h in horarios_ids[0:-3]
        #                 for d in dias_ids)
        #                 == 0)


    # 2 dias para pn 
    # pn = search_profesor_by_nombre(profesores, "pn")
    # if pn is not None:
    #     model.addConstr(gp.quicksum(z_dict[pn.id, d].variable for d in dias_ids) == 2)
    # ------------ ACTUALIZADO EN NUEVAS CONDICIONES ------------

    # max 4 dias para hf
    hf = search_profesor_by_nombre(profesores, "hf")
    if hf is not None:
        model.addConstr(gp.quicksum(z_dict[hf.id, d].variable for d in dias_ids) <= 4)

    # alternar dias para gb (max 4 dias en total)
    gb = search_profesor_by_nombre(profesores, "gb")
    if gb is not None:
        model.addConstr(
            gp.quicksum(z_dict[gb.id, d].variable for d in dias_ids)
            - gp.quicksum(z_dict[gb.id, d].variable * z_dict[gb.id, d+1].variable for d in dias_ids[0:-1])
            >= 2)
    
    # alternar dias para svv
    svv = search_profesor_by_nombre(profesores, "svv")
    if svv is not None:
        model.addConstr(
            gp.quicksum(z_dict[svv.id, d].variable for d in dias_ids)
            - gp.quicksum(z_dict[svv.id, d].variable * z_dict[svv.id, d+1].variable for d in dias_ids[0:-1])
            >= 2)
      


#### (3.3.3) practico despues del teorico

$$ \max_b\{(-b)*u_{mb}\} > \max_b\{(-b)*u_{m'b}\} $$

    para todo par (teorico, practico) =  (m, m')


In [165]:
def constr_teo_prac(model, u_dict):

    """
    Variables:
        teo (list): Array containing IDs of theoretical subjects.
        prac (list): Array containing IDs of practical subjects.
        vars1erHora (dict): Dictionary mapping subject IDs to Gurobi integer variables representing the first hour.
        scaled_u_dict (dict): Dictionary mapping tuples of subject IDs and block IDs to scaled Gurobi integer variables.

    Functions:
        search_materias_by_nombre(materias, nombre): Searches for subjects by name in the given list of subjects.

    Workflow:
    1. Populate `teo` and `vars1erHora` with IDs and variables for theoretical subjects.
    2. Populate `prac` and `vars1erHora` with IDs and variables for practical subjects.
    3. Create scaled variables `scaled_u_dict` for each subject and block combination.
    4. Add constraints to the model to ensure that `vars1erHora` variables are the maximum of the scaled variables.
    5. Add constraints to ensure that the first hour of theoretical subjects is greater than or equal to the first hour of practical subjects if the lengths of `teo` and `prac` are equal.


    """
    teo = []    # array con IDS de materias de teorico
    prac = []    # array con IDS de materias de practico
    vars1erHora = {}
    scaled_u_dict = {}

    for t in [m for m in materias if m.teo_prac == "teo"]:
        teo.append(t.id)
        vars1erHora[t.id] = model.addVar(vtype=GRB.INTEGER, name=str(t)+"_1er_hora")

    for p in [m for m in materias if m.teo_prac == "prac"]:
        prac.append(p.id)
        vars1erHora[p.id] = model.addVar(vtype=GRB.INTEGER, name=str(p)+"_1er_hora")

    L = len(bloques_horario_ids)

    for m in teo + prac:
        for b in range(0, L):
            scaled_u_dict[m, bloques_horario_ids[b]] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"scaled_u")
            model.addConstr(scaled_u_dict[m, bloques_horario_ids[b]] == (L - b) * u_dict[m, bloques_horario_ids[b]].variable)

    for m_id in vars1erHora:
        print(m_id, vars1erHora[m_id])
        model.addConstr(vars1erHora[m_id] ==
                        # gp.max_( (L - b) * u_dict[m_id, bloques_horario_ids[b]].variable
                        gp.max_( scaled_u_dict[m_id, bloques_horario_ids[b]]
                                for b in range(0, L)) )

    if len(teo) == len(prac):
        model.addConstrs(vars1erHora[teo[m]] >= vars1erHora[prac[m]] for m in range(0, len(teo)))


### (3.4) Grupos

#### (3.4.1) definicion de variable "x"

$$ x_{gb} = \sum_{m \in M_g} {u_{mb}} $$

$$ x_{gb} = OR_{m \in M_g} {u_{mb}} $$



In [166]:
def constr_definir_x(model, x_dict, u_dict):

    # SUMA
    # for g in grupos:
    #     gr_mats_ids = materias_grupo_ids(g, materias)
    #     model.addConstrs(x_dict[g.id, b].variable == gp.quicksum(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)

    # OR
    for g in grupos:
        # gr_mats_ids = materias_grupo_ids(g, materias)
        gr_mats_ids = materias_grupo_ids(g, [m for m in materias if not m.electiva]) # solo materias obligatorias
        model.addConstrs(x_dict[g.id, b].variable == gp.or_(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)



#### (3.4.2) evitar horas puente por grupo

$$ \sum_h {x_{gdh}} - \sum_h {x_{gd(h)} · x_{gd(h+1)}} \leq 1 $$
    para todo g, d

In [167]:
def constr_horas_puente_grupos(model, x_dict):

    for g in grupos_reales(grupos):
        model.addConstrs(gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
                    - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
                    <= 1 for d in dias_ids)

### (3.5) Nuevas condiciones / cambios

In [168]:
def constr_cambios(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict):
    
    # cambio 20/11
    # 2 dias para cp
    cp = search_profesor_by_nombre(profesores, "cp")
    if cp is not None:
        model.addConstr(gp.quicksum(z_dict[cp.id, d].variable for d in dias_ids) == 2)

    # cambio 20/11
    # max 3 dias para pn 
    pn = search_profesor_by_nombre(profesores, "pn")
    if pn is not None:
        model.addConstr(gp.quicksum(z_dict[pn.id, d].variable for d in dias_ids) <= 3)

    # cambio 20/11
    # practico ALN (recurse) fijar en viernes
    for alnp in [m.id for m in search_materias_by_nombre(materias, "ALNp")]:
        model.addConstr(v_dict[alnp, 5].variable == 1)

    # cambio 27/11
    # evitar viernes para EG de 4LIC
    for eg in [m.id for m in search_materias_by_nombre(materias, "EG") if "4LIC" in [str(g) for g in m.grupos]]:
        model.addConstr(v_dict[eg, 5].variable == 0)


## (4) Funcion Objetivo

### (4.0) Minimizar cambios

$$ MIN: \sum_{m,b} {(U_{mb}-u_{mb})^2} + \sum_{m,p} {(W_{mp}-w_{mp})^2} + \sum_{p,b} {A_{pb}·(Y_{pb}-y_{pb})^2}$$


In [169]:
def obj_cambios(u_dict, w_dict, y_dict):

    OBJ0_u = gp.QuadExpr()
    OBJ0_w = gp.QuadExpr()
    OBJ0_y = gp.QuadExpr()

    count = 0

    for m in materias:

        for b_id in bloques_horario:
            OBJ0_u += (U_prev[m.id, b_id] - u_dict[m.id, b_id].variable)**2
            count += 1
        
        for p in profesores:
            OBJ0_w += (W_prev[m.id, p.id] - w_dict[m.id, p.id].variable)**2
            count += 1

    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ0_y += A * (Y_prev[p.id, b.id()] - y_dict[p.id, b.id()].variable)**2
            count += 1

    print("obj_cambios terms: ", count)

    return OBJ0_u, OBJ0_w, OBJ0_y


### (4.1) Prioridad horaria de los docentes

$$ MIN: \sum_{m,b,p} {A_{pb}·w_{mp}·u_{mb}} $$


In [170]:
def obj_prioridades(u_dict, w_dict):

    OBJ1 = gp.QuadExpr()
    """
    This script constructs a quadratic expression for an optimization problem using Gurobi's QuadExpr.
    Variables:
        OBJ1 (gp.QuadExpr): A quadratic expression object to accumulate the terms.
        count (int): A counter to keep track of the number of terms added to the quadratic expression.
    Loops:
        The script iterates over all combinations of 'materias' and 'profesores'. For each professor, it further iterates over their 'prioridades'.
    Inner Loop:
        For each priority 'pr' of a professor:
            - 'b' is assigned the 'bloque_horario' of the priority.
            - 'A' is assigned the 'value' of the priority.
            - The quadratic expression 'OBJ1' is updated by adding the product of 'A', the variable corresponding to the combination of 'materia' and 'profesor' from 'w_dict', and the variable corresponding to the combination of 'materia' and 'bloque_horario' from 'u_dict'.
            - The counter 'count' is incremented.
    Final Step:
        The constructed quadratic expression 'OBJ1' is added to the main objective 'OBJ'.
    Output:
        The total count of terms added to the quadratic expression is printed.
    """

    count = 0
    for m in materias:
        for p in profesores:
            for pr in p.prioridades:
                b = pr.bloque_horario
                A = pr.value
                OBJ1 += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable
                count += 1


    print("obj_prioridades terms:", count)

    return OBJ1


LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ MIN: \sum_{p,b} {A_{pb}·y_{pb}} $$

In [171]:
def obj_prioridades_lineal(y_dict):

    OBJ1 = gp.QuadExpr()

    count = 0
    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ1 += A * y_dict[p.id, b.id()].variable
            count += 1

    print("obj_prioridades_lineal terms:", count)

    return OBJ1

### (4.2) Minimizar/Maximizar dias con clase por profesor
$$ MIN/MAX: \sum_{p,d} {z_{pd}} $$

In [172]:
def obj_min_max_dias(z_dict):

    OBJ2 = gp.LinExpr()
    DP = {}

    count = 0
    for p in profesores:
        
        D_p = gp.quicksum(z_dict[p.id,d].variable for d in dias_ids)
        DP[p.id] = D_p

        # filtrar si el profesor prefiere minimizar o maximizar on ninguno
        match p.min_max_dias:
            case "min":
                OBJ2 += D_p
                count += len(dias_ids)
            case "max":
                OBJ2 -= D_p
                count += len(dias_ids)
            case None:
                pass

    print("obj_min_max_dias terms:", count)

    return OBJ2, DP



### (4.3) Minimizar horas puente por grupo (se incluye directamente como restriccion)

$$ MIN: -\sum_{g,d,h} {x_{gd(h)}·x_{gd(h+1)}} $$

In [173]:
def obj_horas_puente_grupos(x_dict):
    OBJ3 = gp.QuadExpr()

    count = 0
    for g in grupos_reales(grupos):
        for d in dias_ids:
            OBJ3 += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
            count += 1

    print("obj_horas_puente_grupos terms:", count)

    return OBJ3


### (4.4) Minimizar superposicion de electivas

$$ MIN:  \sum_{b} \sum_{m, m'}{u_{mb} \times SE_{m,m'} \times u_{m'b}} = 0 $$

para todo par (m, m') de electivas

In [174]:
def obj_superposicion_electivas(u_dict):
    OBJ4 = gp.QuadExpr()

    count = 0
    for m1 in electivas(materias):
        for m2 in electivas(materias):
            for b in bloques_horario_ids:
                OBJ4 += u_dict[m1.id, b].variable * superposicion_electivas[m1.id, m2.id].value * u_dict[m2.id, b].variable
                count += 1

    print("obj_superposicion_electivas terms:", count)

    return OBJ4

# Resolver

### Filtrar (ejecutar desde acá)

In [175]:
# recargar input original

import copy

anios = copy.deepcopy(ANIOS)
dias = copy.deepcopy(DIAS)
horarios = copy.deepcopy(HORARIOS)
bloques_horario = copy.deepcopy(BLOQUES_HORARIO)
profesores = copy.deepcopy(PROFESORES)
grupos = copy.deepcopy(GRUPOS)
materias = copy.deepcopy(MATERIAS)
superposicion = copy.deepcopy(SUPERPOSICION)
superposicion_electivas = copy.deepcopy(SUPERPOSICION_ELECTIVAS)

In [176]:
# # # chequeos de consistencia para asignacion de profesores y materias

# nombres_materias = []
# for m in materias:
#     if str(m) not in nombres_materias:
#         nombres_materias.append(str(m))

# count_asignaciones = {}
# for m in nombres_materias:
#     count_asignaciones[m] = 0

# count = 0
# for m in nombres_materias:
#     length = len(search_materias_by_nombre(materias, m))
#     # print(m, length)
#     count_profs = 0
#     for mi in search_materias_by_nombre(materias, m):
#         count_profs += mi.cantidad_profesores
#     count_asignaciones[m] += count_profs
#     count += count_profs

# print(count)

# count2 = 0
# for p in profesores:
#     # print(p, p.nombre_completo)
#     for l in p.lista_materias:
#         # print('\t', l["nombre_materia"], l["grupos_max"], len([m for m in search_materias_by_nombre(materias, l["nombre_materia"]) if p in m.profesores]))
#         count2 += l["grupos_max"]
#         count_asignaciones[l["nombre_materia"]] -= l["grupos_max"]
#         # for m in search_materias_by_nombre(materias, l["nombre_materia"]):
#             # print(len(mi for mi in))
#             # print('\t\t', m, *m.profesores)
#     # print(p, *[str(l["nombre_materia"]) + ": " + str(l["grupos_max"]) for l in p.lista_materias])

# print(count2)

# print(*[c + ": " + str(count_asignaciones[c]) for c in count_asignaciones], sep="\n")


In [177]:
# chequeo de disponibilidad

# for p in profesores:
#     print(p, p.nombre_completo)
#     for l in p.lista_materias:
#         for mat in search_materias_by_nombre(materias, l["nombre_materia"]):
#             if p not in mat.profesores:
#                 continue
#             print('\t', l["nombre_materia"], l["grupos_max"], mat.carga_horaria)
#             print('\t\thoras:', mat.carga_horaria*l["grupos_max"])
            
#             disp = bloques_horario.values()
#             for t in mat.turnos():
#                 disp = [d for d in disp if t in d.horario.turnos]
            
#             disp = [d for d in disp if d not in p.no_disponible]
#             print('\t\thoras disponibles:', len(disp))
        


In [178]:
# #filtrar por grupos

# grupos = [g for g in grupos if g.anio in [4,5]]

# ms = []
# for m in materias:
#     gs_m = []
#     for g in grupos:
#         if g in m.grupos:
#             gs_m.append(g)

#     if len(gs_m) > 0:
#         m.grupos = gs_m
#         ms.append(m)

# materias = ms


# # ajustar conjunto de profesores y grupos segun las materias filtradas

# ps = []
# gs = []

# for m in materias:
#     print(str(m), ":", [str(g) for g in m.grupos], [str(p) for p in m.profesores], m.carga_horaria, m.cantidad_dias)
#     for p in m.profesores:
#         if p not in ps:
#             ps.append(p)
#     for g in m.grupos:
#         if g not in gs:
#             gs.append(g)

# grupos = gs
# profesores = ps

# anios = [a for a in anios if a in [g.anio for g in grupos]]


In [179]:
# actualizar id arrays

dias_ids = []
for d in dias:
    dias_ids.append(d.id)

horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)

profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)

grupos_ids = []
for g in grupos:
    grupos_ids.append(g.id)

materias_ids = []
for m in materias:
    materias_ids.append(m.id)


### Compilar modelo

In [180]:
# Create a new model
model = gp.Model("timetable")

In [181]:
# variables
from variables import u, v, w, x, y, z
u_dict, v_dict, w_dict, x_dict, y_dict, z_dict = initialize_variables(materias, bloques_horario, dias, profesores, grupos)
create_variables(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict)

u:  12525
v:  835
w:  17368
x:  2175
y:  7800
z:  520


#### Seleccionar restricciones

In [182]:
constr_superposicion(model, u_dict)
constr_carga_horaria(model, u_dict)
constr_dias_materia(model, v_dict)
constr_max_min_horas(model, u_dict, v_dict)

In [183]:
constr_turnos_materia(model, u_dict)
constr_horas_consecutivas(model, u_dict, v_dict)
constr_dias_consecutivos(model, v_dict)
# constr_no_disponible_materia(model, u_dict)

In [184]:
constr_no_disponible_profesor(model, u_dict, w_dict)
# constr_no_disponible_profesor_lineal(model, y_dict)
constr_unica_materia_profesor(model, u_dict, w_dict)
constr_limitar_profesores_materia(model, w_dict)
constr_cantidad_profesores(model, w_dict)
constr_grupos_max_profesor(model, w_dict)

In [185]:
constr_definir_y(model, y_dict, u_dict, w_dict)
constr_definir_z(model, z_dict, y_dict)

In [186]:
constr_cantidad_salones(model, u_dict)
constr_ad_hoc(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict)
constr_teo_prac(model, u_dict)

2 <gurobi.Var *Awaiting Model Update*>
24 <gurobi.Var *Awaiting Model Update*>
25 <gurobi.Var *Awaiting Model Update*>
26 <gurobi.Var *Awaiting Model Update*>
27 <gurobi.Var *Awaiting Model Update*>
32 <gurobi.Var *Awaiting Model Update*>
33 <gurobi.Var *Awaiting Model Update*>
34 <gurobi.Var *Awaiting Model Update*>
35 <gurobi.Var *Awaiting Model Update*>
52 <gurobi.Var *Awaiting Model Update*>
53 <gurobi.Var *Awaiting Model Update*>
56 <gurobi.Var *Awaiting Model Update*>
60 <gurobi.Var *Awaiting Model Update*>
61 <gurobi.Var *Awaiting Model Update*>
75 <gurobi.Var *Awaiting Model Update*>
76 <gurobi.Var *Awaiting Model Update*>
93 <gurobi.Var *Awaiting Model Update*>
117 <gurobi.Var *Awaiting Model Update*>
3 <gurobi.Var *Awaiting Model Update*>
28 <gurobi.Var *Awaiting Model Update*>
29 <gurobi.Var *Awaiting Model Update*>
30 <gurobi.Var *Awaiting Model Update*>
31 <gurobi.Var *Awaiting Model Update*>
36 <gurobi.Var *Awaiting Model Update*>
37 <gurobi.Var *Awaiting Model Update*>
3

In [187]:
constr_definir_x(model, x_dict, u_dict)
constr_horas_puente_grupos(model, x_dict)

In [188]:
constr_cambios(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict)

#### Funcion objetivo 

In [189]:
# funcion objetivo

OBJ = gp.QuadExpr()

OBJ0_u, OBJ0_w, OBJ0_y = obj_cambios(u_dict, w_dict, y_dict)
OBJ0 = OBJ0_u + OBJ0_w + OBJ0_y
OBJ += OBJ0

OBJ1 = obj_prioridades(u_dict, w_dict)
# OBJ += OBJ1

# P2 = 5   # penalidad
# OBJ2, DP = obj_min_max_dias(z_dict)
# OBJ += P2*OBJ2

# OBJ3 = obj_horas_puente_grupos(x_dict)
# # OBJ += OBJ3

P4 = 5  # penalidad
OBJ4 = obj_superposicion_electivas(u_dict)
OBJ += P4*OBJ4

model.setObjective(OBJ, GRB.MINIMIZE)

obj_cambios terms:  37693
obj_prioridades terms: 1302600
obj_superposicion_electivas terms: 39675


### Optimizar

In [190]:
model.setParam("TimeLimit", 60*60*2)
# model.Params.MIPGap = 1/100
# model.Params.PoolSolutions = 10 # Number of solutions to find
# model.Params.PoolSearchMode = 0 # 0, 1, 2

model.optimize()

# nSolutions = model.SolCount
# print(f"Number of solutions found: {nSolutions}")

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 30350 rows, 61284 columns and 101905 nonzeros
Model fingerprint: 0xe79d9e6e
Model has 37093 quadratic objective terms
Model has 36805 quadratic constraints
Model has 10531 general constraints
Variable types: 0 continuous, 61284 integer (41223 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+00, 2e+00]
Presolve removed 23345 rows and 54275 columns
Presolve time: 1.40s
Presolved: 22579 rows, 11805 columns, 66629 nonzeros
Variable types: 0 continuous, 11805 integer (11593 

In [191]:
# # Function to get variable values for a specific solution
# def get_solution_values(model, solution_number):
#     model.setParam("SolutionNumber", solution_number)
#     return {var.varName: round(var.x) for var in model.getVars()}

# # Get the number of solutions
# num_solutions = model.SolCount

# # Store variable values for each solution
# solutions = []
# for i in range(num_solutions):
#     solutions.append(get_solution_values(model, i))

# # Compare solutions to find differences
# differences = {}
# for var_name in solutions[0].keys():
#     values = [solution[var_name] for solution in solutions]
#     if len(set(values)) > 1:  # If there are different values for this variable
#         differences[var_name] = values

# # Print differences
# for var_name, values in differences.items():
#     print(f"Variable {var_name} has different values: {values}")

#### Analizar no-factibilidad 

In [192]:
# if model.Status == GRB.INFEASIBLE:
#     model.computeIIS()
#     model.write('iismodel.ilp')

#     # Print out the IIS constraints and variables
#     print('\nThe following constraints and variables are in the IIS:')
#     for c in model.getConstrs():
#         if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

#     for v in model.getVars():
#         if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
#         if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

### Imprimir horarios

In [193]:
model.setParam("SolutionNumber", 0)
model.update()


if not model.Status == GRB.INFEASIBLE:
    print('Obj: %g' % model.ObjVal)
    print('OBJ0: %g' % OBJ0.getValue())
    print('\tOBJ0_u: %g' % OBJ0_u.getValue())
    print('\tOBJ0_w: %g' % OBJ0_w.getValue())
    print('\tOBJ0_y: %g' % OBJ0_y.getValue())
    print('OBJ1: %g' % OBJ1.getValue())
    # print('OBJ2: %g' % OBJ2.getValue())
    print('OBJ4: %g' % OBJ4.getValue())
    print('=============================================================')
    print_timetable(dias, horarios, u_dict, w_dict, grupos, anios)


Obj: 293
OBJ0: 33
	OBJ0_u: 18
	OBJ0_w: 0
	OBJ0_y: 15
OBJ1: 889
OBJ4: 52

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  11
			lun		mar		mie		jue		vie
··········································································································
08:00-08:50		AL		Prog1		---		---		Prog1
08:50-09:40		AL		Prog1		---		---		Prog1
09:50-10:40		AL		Prog1		---		TIngA		ALp
10:40-11:30		TIngB		AM1		AL		TIngA		ALp
11:40-12:30		TIngB		AM1		AL		AM1		AM1p
12:30-13:20		TIngB		AM1		---		AM1		AM1p

 Grupo:  12
			lun		mar		mie		jue		vie
··········································································································
08:00-08:50		Prog1		---		Prog1		---		AM1p
08:50-09:40		Prog1		---		Prog1		---		AM1p
09:50-10:40		AL		TIngB		Prog1		AL		TIngA
10:40-11:30		AL		TIngB		AM1		AL		TIngA
11:40-12:30		AM1		TIngB		AM1		AL		ALp
12:30-13:20		AM1		---		AM1		---		ALp

 Grupo:  13
			lun		mar		mie		jue		vie
·····

#### Mostrar cambios

In [194]:
if not model.Status == GRB.INFEASIBLE:
    for m in materias:

        for b_id in bloques_horario:
            prev = U_prev[m.id, b_id]
            new = round(u_dict[m.id, b_id].variable.x)
            if (prev - new)**2 == 1:
                print("u", m, bloques_horario[b_id], "X" if new < prev else "<-")
        
        for p in profesores:
            prev = W_prev[m.id, p.id]
            new = round(w_dict[m.id, p.id].variable.x)
            if (prev - new)**2 == 1:
                print("w", m, p, "X" if new < prev else "<-")

    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            prev = Y_prev[p.id, b.id()]
            new = round(y_dict[p.id, b.id()].variable.x)
            if (prev - new)**2 == 1:
                print("y", p, b, "X" if new < prev else "<-")

u IAL mie_20:40-21:30 X
u IAL mie_21:40-22:30 X
u IAL jue_18:00-18:50 <-
u IAL jue_18:50-19:40 <-
u PComp mie_20:40-21:30 <-
u PComp mie_21:40-22:30 <-
u PComp jue_18:00-18:50 X
u PComp jue_18:50-19:40 X
u EG mie_21:40-22:30 <-
u EG vie_21:40-22:30 X
u ALN mie_14:10-15:00 <-
u ALN mie_15:10-16:00 <-
u ALN vie_14:10-15:00 X
u ALN vie_15:10-16:00 X
u ALNp mie_14:10-15:00 X
u ALNp mie_15:10-16:00 X
u ALNp vie_14:10-15:00 <-
u ALNp vie_15:10-16:00 <-
y cp mie_20:40-21:30 X
y cp mie_21:40-22:30 X
y cp jue_18:00-18:50 <-
y cp jue_18:50-19:40 <-
y icc1 mie_21:40-22:30 <-
y icc1 vie_21:40-22:30 X
y pn mie_20:40-21:30 <-
y pn mie_21:40-22:30 <-
y pn jue_18:00-18:50 X
y pn jue_18:50-19:40 X


#### Imprimir para Excel

In [195]:
# # # IMPRIMIR PARA COPIAR EN HOJA DE EXCEL
if not model.Status == GRB.INFEASIBLE:
    print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, anios, show_profs=True)


 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  11
	lun	mar	mie	jue	vie
08:00-08:50	AL[mp]	Prog1[vt]	---	---	Prog1[vt]
08:50-09:40	AL[mp]	Prog1[vt]	---	---	Prog1[vt]
09:50-10:40	AL[mp]	Prog1[vt]	---	TIngA[as]	ALp[jf]
10:40-11:30	TIngB[ad]	AM1[am]	AL[mp]	TIngA[as]	ALp[jf]
11:40-12:30	TIngB[ad]	AM1[am]	AL[mp]	AM1[am]	AM1p[ps]
12:30-13:20	TIngB[ad]	AM1[am]	---	AM1[am]	AM1p[ps]

 Grupo:  12
	lun	mar	mie	jue	vie
08:00-08:50	Prog1[p1a]	---	Prog1[p1a]	---	AM1p[ps]
08:50-09:40	Prog1[p1a]	---	Prog1[p1a]	---	AM1p[ps]
09:50-10:40	AL[svv]	TIngB[ad]	Prog1[p1a]	AL[svv]	TIngA[as]
10:40-11:30	AL[svv]	TIngB[ad]	AM1[lg]	AL[svv]	TIngA[as]
11:40-12:30	AM1[lg]	TIngB[ad]	AM1[lg]	AL[svv]	ALp[jf]
12:30-13:20	AM1[lg]	---	AM1[lg]	---	ALp[jf]

 Grupo:  13
	lun	mar	mie	jue	vie
08:00-08:50	TIngB[ad]	---	---	---	ALp[jf]
08:50-09:40	TIngB[ad]	TIngA[as]	---	---	ALp[jf]
09:50-10:40	TIngB[ad]	TIngA[as]	---	AM1[gc]	AL[ds]
10:40-11:30	Prog1[p1c]	AM1

In [196]:
if not model.Status == GRB.INFEASIBLE:
    print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)


 Profesor:  aca Andrés Cardozo
	lun	mar	mie	jue	vie
08:00-08:50	---	---	---	---	MIE
08:50-09:40	---	---	---	---	MIE
09:50-10:40	---	---	---	---	MIE

 Profesor:  ad Andrea Durlacher
	lun	mar	mie	jue	vie
08:00-08:50	TIngB	---	---	---	---
08:50-09:40	TIngB	---	---	---	---
09:50-10:40	TIngB	TIngB	---	---	---
10:40-11:30	TIngB	TIngB	---	---	---
11:40-12:30	TIngB	TIngB	---	---	---
12:30-13:20	TIngB	---	---	---	---
14:10-15:00	---	TIngB	---	---	---
15:10-16:00	---	TIngB	---	---	---
16:10-17:00	---	TIngB	---	---	---

 Profesor:  adc Adrián Carballo
	lun	mar	mie	jue	vie
18:50-19:40	---	MotCI	---	---	---
19:50-20:40	---	MotCI	---	---	---
20:40-21:30	---	MotCI	---	---	---

 Profesor:  adr Alejandro Draper
	lun	mar	mie	jue	vie
18:00-18:50	---	InfrInf	---	InfrInf	---
18:50-19:40	---	InfrInf	---	InfrInf	---

 Profesor:  am Andrea Mesa
	lun	mar	mie	jue	vie
10:40-11:30	---	AM1	---	---	---
11:40-12:30	---	AM1	---	AM1	---
12:30-13:20	---	AM1	---	AM1	---
14:10-15:00	---	AM2	---	AM2	---
15:10-16:00	---	A

### Estadísticas de resultados

#### Promedios de prioridad

In [197]:
# CARGA_HORARIA_TOTAL = gp.quicksum(m.carga_horaria for m in materias)
CARGA_HORARIA_TOTAL = gp.quicksum(u_dict[m, b].variable * w_dict[m, p].variable for p in profesores_ids for m in materias_ids for b in bloques_horario_ids)

if not model.Status == GRB.INFEASIBLE:
    print(OBJ1.getValue())
    print("Promedio general de prioridad horaria: ", OBJ1.getValue()/CARGA_HORARIA_TOTAL.getValue())
          

# print("Casos de materias con dias consecutivos: ", int(OBJ2.getValue()/10))


889.0
Promedio general de prioridad horaria:  1.3890625


In [198]:
# promedio por profesor:
"""
This script calculates and prints the average workload per professor based on their assigned priorities and time blocks.
Variables:
    profesores (list): List of professor objects, each containing their priorities.
    y_dict (dict): Dictionary mapping professor IDs and time block IDs to Gurobi variables.
    bloques_horario_ids (list): List of time block IDs.
For each professor in the list `profesores`, the script performs the following steps:
1. Initializes a quadratic expression `OBJ_p` to accumulate the weighted priorities.
2. Iterates over the professor's priorities to update `OBJ_p` based on the priority value and corresponding Gurobi variable.
3. Calculates the total workload `CARGA_HORARIA_p` for the professor by summing the relevant Gurobi variables.
4. If the total workload is non-zero, it prints the professor's name, total workload, and the rounded average priority value per workload unit.
5. If the total workload is zero, it prints the professor's name and indicates that they have no workload.

"""
if not model.Status == GRB.INFEASIBLE:
    for p in profesores:

        OBJ_p = gp.QuadExpr()
        # for m in materias:
        #     for pr in p.prioridades:
        #             b = pr.bloque_horario
        #             A = pr.value
        #             OBJ_p += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable

        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ_p += A * y_dict[p.id, b.id()].variable
        
        # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()
        CARGA_HORARIA_p = round(gp.quicksum(y_dict[p.id, b].variable for b in bloques_horario_ids).getValue())

        if CARGA_HORARIA_p != 0:
            print(str(p), "\t", CARGA_HORARIA_p, "\t", round(round(OBJ_p.getValue())/CARGA_HORARIA_p,3))
        else:
            print(str(p), "sin carga horaria")

    


aca 	 3 	 1.0
ad 	 12 	 1.0
adc 	 3 	 1.0
adr 	 4 	 1.0
am 	 12 	 1.167
ame 	 6 	 1.667
an 	 4 	 1.0
anp 	 3 	 1.333
ap 	 5 	 1.0
apa 	 4 	 1.25
apr 	 5 	 1.8
ar 	 7 	 1.286
arqap 	 5 	 1.0
as 	 8 	 1.0
cm 	 7 	 1.0
cp 	 10 	 2.0
cr 	 4 	 1.0
db 	 10 	 1.8
dc 	 7 	 1.286
dd 	 8 	 1.0
df 	 4 	 2.0
dj 	 8 	 1.0
dl 	 15 	 1.867
dm 	 4 	 1.0
ds 	 11 	 2.273
ep 	 8 	 1.0
epe 	 6 	 2.0
er 	 6 	 1.0
es 	 3 	 1.0
fb 	 5 	 3.0
fr 	 3 	 1.333
gaa 	 5 	 1.2
gb 	 11 	 1.0
gba 	 4 	 1.25
gc 	 12 	 1.167
gca 	 4 	 1.0
gd 	 4 	 1.25
gdo 	 4 	 2.0
gg 	 4 	 1.0
gp 	 5 	 1.2
gq 	 5 	 1.8
guc 	 10 	 1.1
hf 	 10 	 2.2
hr 	 5 	 1.0
ht 	 5 	 2.4
icc1 	 7 	 1.0
icc2 	 12 	 1.0
icc3 	 2 	 1.0
jcc 	 2 	 1.0
jd 	 10 	 1.6
jds 	 7 	 1.429
jf 	 8 	 1.0
jg 	 2 	 3.0
jj 	 4 	 1.0
jk 	 14 	 1.571
jl 	 4 	 1.0
jm 	 7 	 1.0
jmi 	 4 	 1.25
jpa 	 5 	 2.6
jpe 	 4 	 1.0
jpf 	 10 	 1.4
jpi 	 4 	 1.75
lc 	 2 	 1.0
lg 	 9 	 1.889
ma 	 2 	 1.5
mb 	 5 	 1.0
mbo 	 2 	 1.0
mc 	 5 	 1.6
md 	 3 	 3.0
mf 	 5 	 1.2
mg 	 2 	 2.0
mjc 

In [199]:
# # DIAS CLASE POR PROFESOR

# print("DIAS CON CLASE")
# for p in profesores:
#     if p.min_max_dias is not None:
#         print(str(p), ": ", round(DP[p.id].getValue()), p.min_max_dias)

In [200]:
# PROFESORES CON DISPONIBILIDAD LIMITADA

# for p in profesores:
        
#     suma_pr = 0
#     carga_horaria = 0
    
#     for pr in p.prioridades:
#         suma_pr = suma_pr + 1 if pr.value > 0 else suma_pr

#     for m in p.lista_materias:
#         try:
#             horas = search_materias_by_nombre(materias, m["nombre_materia"])[0].carga_horaria
#             grupos_max = m["grupos_max"]
#             carga_horaria += horas*grupos_max
#         except:
#             continue

#     if carga_horaria > 0:
#         margen = int((suma_pr/carga_horaria-1)*100)

#         if margen < 250:
#             print('----------------------')
#             print(str(p), p.nombre_completo)
#             print("disponibilidad:", suma_pr)
#             print("carga horaria:", carga_horaria)
#             print("margen:", margen, "%")




In [201]:
# HORAS PUENTE

# HP = gp.QuadExpr()
# for g in grupos_reales:
#     HPg = gp.QuadExpr()
#     for d in dias_ids:
#         HPg += gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
#         HPg += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
#         HPg += - 1
#     # print("horas puente grupo", str(g), ": ", HPg.getValue())
#     HP += HPg

# print("Total de horas puente: ", HP.getValue())

#### Distribucion de prioridades

In [202]:
# distribucion de prioridades
"""
This script calculates and prints the distribution of priorities and their averages for a given set of professors and their time block priorities.
Variables:
    promedios_prioridad (dict): A dictionary to store the average priority values for each priority level.
    valores_prioridad (dict): A dictionary to store the count of each priority level.
    profesores (list): A list of professor objects, each containing their priority information.
    y_dict (dict): A dictionary containing the decision variables for the optimization model.
Process:
1. Initialize dictionaries to store priority averages and counts.
2. Iterate over each professor and their priorities:
    - Skip if the priority value is 0.
    - Increment the count for the given priority value.
    - Update the average priority value based on the decision variable.
3. Print the count and percentage distribution of each priority level.
4. Print the average priority values and their percentage distribution.

"""

if not model.Status == GRB.INFEASIBLE:
    promedios_prioridad = {}
    valores_prioridad = {}
    for i in range(1,4):
        promedios_prioridad[i] = 0
        valores_prioridad[i] = 0

    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            if A == 0:
                continue
            valores_prioridad[A] += 1
            # for m in materias:
            #     promedios_prioridad[A] += round(w_dict[m.id, p.id].variable.X * u_dict[m.id, b.id()].variable.X)
            promedios_prioridad[A] += round(y_dict[p.id, b.id()].variable.X)

    print(valores_prioridad)
    total = np.sum([valores_prioridad[i] for i in valores_prioridad])
    print(*[str(i) + ": " + str(round(valores_prioridad[i]/total*100))+"%" for i in valores_prioridad], sep=", ")

    print(promedios_prioridad)
    total = np.sum([promedios_prioridad[i] for i in promedios_prioridad])
    if total != 0:
        print(*[str(i) + ": " + str(round(promedios_prioridad[i]/total*100))+"%" for i in promedios_prioridad], sep=", ")
        
        # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()

        # if CARGA_HORARIA_p != 0:
        #     print(str(p), ": ", OBJ_p.getValue()/CARGA_HORARIA_p)
        # else:
        #     print(str(p), "sin carga horaria")

{1: 1746, 2: 394, 3: 560}
1: 65%, 2: 15%, 3: 21%
{1: 461, 2: 85, 3: 76}
1: 74%, 2: 14%, 3: 12%


#### Resultados superposicion de electivas

In [203]:
res_super_electivas = {}

for g in grupos:
    ms = [e for e in electivas(materias) if g in [g for g in e.grupos]]
    if len(ms) > 0:
        print(g)
        res_super_electivas[g.id] = {}
    for m1 in ms:
        # print('\t', m1)
        for m2 in [m for m in ms if m1.nombre != m.nombre]:
            sum = np.sum([round(u_dict[m1.id, b].variable.x) * round(u_dict[m2.id, b].variable.x) for b in bloques_horario_ids])
            # print('\t\t', m2, sum)
            if (m1.nombre, m2.nombre) not in res_super_electivas[g.id]:
                res_super_electivas[g.id][m1.nombre, m2.nombre] = sum
            else:
                res_super_electivas[g.id][m1.nombre, m2.nombre] += sum
    
    nombres = set(m.nombre for m in ms)
    for m1 in nombres:
        print('\t', m1)
        for m2 in [m for m in nombres if m1 != m]:
            print('\t\t', m2, res_super_electivas[g.id][m1, m2])


4CIV
	 Horm2
		 DIPyU 0
		 Puentes 0
		 DPav 0
	 DIPyU
		 Horm2 0
		 Puentes 0
		 DPav 1
	 Puentes
		 Horm2 0
		 DIPyU 0
		 DPav 2
	 DPav
		 Horm2 0
		 DIPyU 1
		 Puentes 2
4IND
	 GAmb
		 MdProc 0
		 TecFP 1
	 MdProc
		 GAmb 0
		 TecFP 0
	 TecFP
		 GAmb 1
		 MdProc 0
4LIC
	 DBD3
		 ApMov 3
		 RPA 2
		 BPMN 2
		 EthHck 2
	 ApMov
		 DBD3 3
		 RPA 2
		 BPMN 0
		 EthHck 0
	 RPA
		 DBD3 2
		 ApMov 2
		 BPMN 2
		 EthHck 2
	 BPMN
		 DBD3 2
		 ApMov 0
		 RPA 2
		 EthHck 3
	 EthHck
		 DBD3 2
		 ApMov 0
		 RPA 2
		 BPMN 3
5CIV
	 StUp1
		 AgroN 2
		 DPav 0
		 DisEst 2
		 Puentes 1
		 DIPyU 0
	 AgroN
		 StUp1 2
		 DPav 0
		 DisEst 0
		 Puentes 2
		 DIPyU 0
	 DPav
		 StUp1 0
		 AgroN 0
		 DisEst 0
		 Puentes 2
		 DIPyU 1
	 DisEst
		 StUp1 2
		 AgroN 0
		 DPav 0
		 Puentes 0
		 DIPyU 0
	 Puentes
		 StUp1 1
		 AgroN 2
		 DPav 2
		 DisEst 0
		 DIPyU 0
	 DIPyU
		 StUp1 0
		 AgroN 0
		 DPav 1
		 DisEst 0
		 Puentes 0
5IND
	 InstEl
		 AgroN 1
		 MotCI 0
		 StUp1 2
		 TecFP 0
	 AgroN
		 InstEl 1
		 MotCI 

#### Conjuntos de electivas compatibles

In [204]:
# CONJUNTOS DE ELECTIVAS COMPATIBLES

import itertools

def find_subsets(ms, res_super_electivas_g):
    # Generate all possible subsets of the array ms
    subsets = []
    for r in range(1, len(ms) + 1):
        subsets.extend(list(itertools.combinations(ms, r)))
    
    # Filter subsets to only include those with dictionary value 0 between all of them
    valid_subsets = []
    for subset in subsets:
        valid = True
        for m1, m2 in itertools.combinations(subset, 2):
            if res_super_electivas_g.get((m1, m2), None) != 0:
                valid = False
                break
        if valid:
            valid_subsets.append(subset)
    
    return valid_subsets

# for g in grupos:
#     ms = set(e.nombre for e in elecs if g in [g for g in e.grupos])
#     if len(ms) > 0:
#         print(g)
#         for s in find_subsets(ms, res_super_electivas[g.id]):
#             print('\t', [m for m in s])

for g in grupos:
    ms = set(e.nombre for e in electivas(materias) if g in [g for g in e.grupos])
    if len(ms) > 0:
        print(g)
        subsets = find_subsets(ms, res_super_electivas[g.id])
        for m in ms:
            s_max = [m]
            for s in [s for s in subsets if m in s]:
                if len(s) > len(s_max):
                    s_max = s
            print('\t', m, [mi for mi in s if mi != m])

        # for s in subsets:
        #     print('\t', [m for m in s])


4CIV
	 Horm2 ['DIPyU', 'Puentes']
	 DIPyU ['Horm2', 'Puentes']
	 Puentes ['Horm2', 'DIPyU']
	 DPav ['Horm2']
4IND
	 GAmb ['MdProc']
	 MdProc ['TecFP']
	 TecFP ['MdProc']
4LIC
	 DBD3 []
	 ApMov ['EthHck']
	 RPA []
	 BPMN ['ApMov']
	 EthHck ['ApMov']
5CIV
	 StUp1 ['DIPyU']
	 AgroN ['DisEst', 'DIPyU']
	 DPav ['AgroN', 'DisEst']
	 DisEst ['Puentes', 'DIPyU']
	 Puentes ['DisEst', 'DIPyU']
	 DIPyU ['DisEst', 'Puentes']
5IND
	 InstEl ['MotCI', 'TecFP']
	 AgroN ['MotCI', 'TecFP']
	 MotCI ['StUp1', 'TecFP']
	 StUp1 ['MotCI', 'TecFP']
	 TecFP ['MotCI', 'StUp1']
5TEL
	 DBD3 ['UIUX', 'AgroN', 'QC']
	 ApMov ['UIUX', 'AgroN', 'EthHck', 'QC']
	 TestQA ['UIUX', 'GSInf', 'EthHck', 'QC']
	 UIUX ['AgroN', 'GSInf', 'EthHck', 'QC']
	 AgroN ['UIUX', 'GSInf', 'EthHck', 'QC']
	 RPA ['UIUX', 'AgroN', 'GSInf', 'QC']
	 StUp1 ['ApMov', 'TestQA', 'UIUX', 'EthHck']
	 GSInf ['UIUX', 'AgroN', 'EthHck', 'QC']
	 BPMN ['UIUX', 'AgroN', 'GSInf', 'QC']
	 EthHck ['UIUX', 'AgroN', 'GSInf', 'QC']
	 QC ['UIUX', 'AgroN', 'GSIn

In [205]:
# def include_mat(set, m):
#     include = True
#     for mprev in set:
#         if m != mprev and res_super_electivas[m.id, mprev.id] != 0:
#             include = False
#             break
#     return include

# def update_set(set, ms):
#     sets = []
    
#     if len(ms) == 0:
#         return [set]
    
#     # if len(ms) == 1:
#     for mi in ms:
#         if include_mat(set, mi):
#             sets += update_set(set + [mi], ms[1:])
#             print(*ms[1:])
#         else:
#             sets += set

#     # else:
#     #     if include_mat(set, mi):
#     #         sets += update_set(set + [m1], ms[1:])
#     #     sets += update_set(set, ms[1:])                
#     return sets
        
        
# for g in grupos:
#     ms = [e for e in elecs if g in [g for g in e.grupos]]
#     for m1 in ms:
#         sets_posibles += update_set([m1], [m for m in ms if m != m1])


#     # update_set(sets_posibles, ms)



#### Ocupacion de salones

In [206]:
# nivel de ocupacion
if not model.Status == GRB.INFEASIBLE:
    niveles = {}

    for b_id in bloques_horario:
        salones_ocupados = int(gp.quicksum(u_dict[m, b_id].variable for m in materias_ids).getValue())
        nivel = int(salones_ocupados/K*100)
        # print(str(bloques_horario[b_id]), ": ", salones_ocupados,",", str(nivel) + "%")
        niveles[b_id] = nivel

    print_timetable_salones(dias, horarios, niveles)

		lun	mar	mie	jue	vie
··········································································································
08:00-08:50	61%	69%	53%	53%	100%
08:50-09:40	76%	76%	61%	53%	100%
09:50-10:40	76%	69%	61%	76%	100%
10:40-11:30	76%	69%	76%	69%	76%
11:40-12:30	76%	76%	76%	69%	76%
12:30-13:20	69%	61%	46%	38%	61%
14:10-15:00	15%	23%	15%	23%	15%
15:10-16:00	53%	61%	46%	61%	53%
16:10-17:00	53%	76%	61%	61%	53%
17:00-17:50	61%	69%	61%	38%	61%
18:00-18:50	84%	84%	92%	84%	61%
18:50-19:40	84%	76%	100%	76%	84%
19:50-20:40	76%	76%	92%	69%	92%
20:40-21:30	61%	53%	69%	53%	69%
21:40-22:30	23%	23%	23%	15%	7%


In [207]:
# for u_i in u_dict:
#     u = u_dict[u_i]
#     print(str(u) + " = ", u.variable.X)

# for v_i in v_dict:
#     v = v_dict[v_i]
#     print(str(v) + " = ", v.variable.X)

# for w_i in w_dict:
#     w = w_dict[w_i]
#     print(str(w) + " = ", w.variable.X)


In [208]:
model.write('timetable_opt.lp')
model.write('timetable_opt.mps')
